<a href="https://colab.research.google.com/github/YJ-Sun-1994/TheImprovedNeuralNetworkModelInHumorDetectionWithTraditionalHumorTheory/blob/main/%E2%80%9CCopy_of_code_ipynb%EF%BC%88%E5%89%AF%E6%9C%AC%EF%BC%89%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initialization

##Connect Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Install Transformers

In [ ]:
!pip install transformers
!pip install utils

##Import Module

In [ ]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from tokenizers import AddedToken
from transformers import RobertaTokenizer
import math
import csv
import re
import os
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn import model_selection
from sklearn import metrics
import transformers
import tokenizers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import RobertaConfig, RobertaModel
#from transformers.modeling_roberta import RobertaClassificationHead
from sklearn.model_selection import train_test_split
from tqdm.autonotebook import tqdm
import utils
from transformers import RobertaConfig, RobertaModel, RobertaForSequenceClassification, RobertaTokenizer
from transformers import BertConfig, BertModel, BertForSequenceClassification, BertTokenizer
from transformers import AdamW
import transformers
import torch
from transformers import get_linear_schedule_with_warmup
import zipfile
import random

##SetSeed

In [ ]:
def fix_seed(seed=1):
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)
fix_seed()

##Configuration&Tokenizer

In [ ]:
configuration = RobertaConfig()
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=True)

#Data

##Task1

###No Dev.csv

In [ ]:
dev_type = 'No_dev'
import csv
import re
def readTask1Data(data_address):
  data_id = list()
  data_mask = list()
  data_orginal = list()
  data_replaced = list()
  data_grades = list()
  data_meanGrade = list()
  data_editword = list()
  data_orginalword = list()
  flag = 0
  p = r"(?<=<).+?(?=/>)" 
  with open(data_address, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      data_id.append(row[0])
      line = row[1]
      while '  ' in line:
        line.replace('  ',' ')
      line = line.replace('/ >','/>')
      line = line.replace('< ','<')
      line = line.replace(' />','/>')
      data_orginalword.append(re.findall(p, line))
      data_orginal_temp = line.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal.append(data_orginal_temp)
      data_mask_temp = re.sub(p, 'edit', line)
      if '<edit/>' not in data_mask_temp:
        print(data_mask_temp)
      data_mask.append(data_mask_temp)
      data_replaced_temp = data_mask_temp.replace('<edit/>',row[2])
      data_replaced.append(data_replaced_temp)
      data_grades.append(row[3])
      data_meanGrade.append([float(row[4])])
      data_editword.append(row[2])
  return  data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test_mask, test_orginal, test_replaced, test_grades, test_meanGrade, test_editword, test_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
def splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, valid_num=0.3):
  train_id = list()
  train_mask = list()
  train_orginal = list()
  train_replaced = list()
  train_grades = list()
  train_meanGrade = list()
  train_editword = list()
  train_orginalword = list()
  val_id = list()
  val_mask = list()
  val_orginal = list()
  val_replaced = list()
  val_grades = list()
  val_meanGrade = list()
  val_editword = list()
  val_orginalword = list()
  temp_list = list()
  for num in range(len(data_id)):
    temp_list.append([data_id[num], data_mask[num], data_orginal[num], data_replaced[num], data_grades[num], data_meanGrade[num], data_editword[num], data_orginalword[num]])
  train_set,valid_set = train_test_split(temp_list,test_size = valid_num)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in train_set:
    train_id.append(set_id_temp)
    train_mask.append(set_mask_temp)
    train_orginal.append(set_orginal_temp)
    train_replaced.append(set_replaced_temp)
    train_grades.append(set_grades_temp)
    train_meanGrade.append(set_meanGrade_temp)
    train_editword.append(set_editword_temp)
    train_orginalword.append(set_orginalword_temp)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in valid_set:
    val_id.append(set_id_temp)
    val_mask.append(set_mask_temp)
    val_orginal.append(set_orginal_temp)
    val_replaced.append(set_replaced_temp)
    val_grades.append(set_grades_temp)
    val_meanGrade.append(set_meanGrade_temp)
    val_editword.append(set_editword_temp)
    val_orginalword.append(set_orginalword_temp)
  return train_id, train_mask, train_orginal, train_replaced, train_grades, train_meanGrade, train_editword, train_orginalword, val_id,val_mask,val_orginal,val_replaced,val_grades,val_meanGrade,val_editword,val_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, val_id, val_mask, val_orginal, val_replaced, val_grades, val_meanGrade, val_editword, val_orginalword = splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword)

###With Dev.csv

In [ ]:
dev_type = 'with_dev'
import csv
import re
def readTask1Data(data_address):
  data_id = list()
  data_mask = list()
  data_orginal = list()
  data_replaced = list()
  data_grades = list()
  data_meanGrade = list()
  data_editword = list()
  data_orginalword = list()
  flag = 0
  p = r"(?<=<).+?(?=/>)" 
  with open(data_address, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      data_id.append(row[0])
      line = row[1]
      while '  ' in line:
        line.replace('  ',' ')
      line = line.replace('/ >','/>')
      line = line.replace('< ','<')
      line = line.replace(' />','/>')
      data_orginalword.append(re.findall(p, line))
      data_orginal_temp = line.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal.append(data_orginal_temp)
      data_mask_temp = re.sub(p, 'edit', line)
      if '<edit/>' not in data_mask_temp:
        print(data_mask_temp)
      data_mask.append(data_mask_temp)
      data_replaced_temp = data_mask_temp.replace('<edit/>',row[2])
      data_replaced.append(data_replaced_temp)
      data_grades.append(row[3])
      data_meanGrade.append([float(row[4])])
      data_editword.append(row[2])
  return  data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test_mask, test_orginal, test_replaced, test_grades, test_meanGrade, test_editword, test_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
val_id, val_mask, val_orginal, val_replaced, val_grades, val_meanGrade, val_editword, val_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/dev.csv')
data_id = data_id + val_id
data_mask = data_mask + val_mask
data_orginal = data_orginal + val_orginal
data_replaced = data_replaced + val_replaced
data_grades = data_grades + val_grades
data_meanGrade = data_meanGrade + val_meanGrade
data_editword = data_editword + val_editword
data_orginalword = data_orginalword + val_orginalword
def splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, valid_num=0.3):
  train_id = list()
  train_mask = list()
  train_orginal = list()
  train_replaced = list()
  train_grades = list()
  train_meanGrade = list()
  train_editword = list()
  train_orginalword = list()
  val_id = list()
  val_mask = list()
  val_orginal = list()
  val_replaced = list()
  val_grades = list()
  val_meanGrade = list()
  val_editword = list()
  val_orginalword = list()
  temp_list = list()
  for num in range(len(data_id)):
    temp_list.append([data_id[num], data_mask[num], data_orginal[num], data_replaced[num], data_grades[num], data_meanGrade[num], data_editword[num], data_orginalword[num]])
  train_set,valid_set = train_test_split(temp_list,test_size = valid_num)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in train_set:
    train_id.append(set_id_temp)
    train_mask.append(set_mask_temp)
    train_orginal.append(set_orginal_temp)
    train_replaced.append(set_replaced_temp)
    train_grades.append(set_grades_temp)
    train_meanGrade.append(set_meanGrade_temp)
    train_editword.append(set_editword_temp)
    train_orginalword.append(set_orginalword_temp)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in valid_set:
    val_id.append(set_id_temp)
    val_mask.append(set_mask_temp)
    val_orginal.append(set_orginal_temp)
    val_replaced.append(set_replaced_temp)
    val_grades.append(set_grades_temp)
    val_meanGrade.append(set_meanGrade_temp)
    val_editword.append(set_editword_temp)
    val_orginalword.append(set_orginalword_temp)
  return train_id, train_mask, train_orginal, train_replaced, train_grades, train_meanGrade, train_editword, train_orginalword, val_id,val_mask,val_orginal,val_replaced,val_grades,val_meanGrade,val_editword,val_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, val_id, val_mask, val_orginal, val_replaced, val_grades, val_meanGrade, val_editword, val_orginalword = splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword)

##Task2

###No Dev.csv

In [ ]:
import csv
import re
def readTask1Data(data_address):
  data_id = list()
  data_mask1 = list()
  data_orginal1 = list()
  data_replaced1 = list()
  data_grades1 = list()
  data_meanGrade1 = list()
  data_editword1 = list()
  data_orginalword1 = list()
  data_mask2 = list()
  data_orginal2 = list()
  data_replaced2 = list()
  data_grades2 = list()
  data_meanGrade2 = list()
  data_editword2 = list()
  data_orginalword2 = list()
  data_label = list()
  flag = 0
  p = r"(?<=<).+?(?=/>)" 
  with open(data_address, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      data_id.append(row[0])
      line1 = row[1]
      while '  ' in line1:
        line1.replace('  ',' ')
      line1 = line1.replace('/ >','/>')
      line1 = line1.replace('< ','<')
      line1 = line1.replace(' />','/>')
      data_orginalword1.append(re.findall(p, line1))
      data_orginal_temp1 = line1.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal1.append(data_orginal_temp1)
      data_mask_temp1 = re.sub(p, 'edit', line1)
      if '<edit/>' not in data_mask_temp1:
        print(data_mask_temp)
      data_mask1.append(data_mask_temp1)
      data_replaced_temp1 = data_mask_temp1.replace('<edit/>',row[2])
      data_replaced1.append(data_replaced_temp1)
      data_grades1.append(row[3])
      data_meanGrade1.append([float(row[4])])
      data_editword1.append(row[2])


      line2 = row[5]
      while '  ' in line2:
        line2.replace('  ',' ')
      line2 = line2.replace('/ >','/>')
      line2 = line2.replace('< ','<')
      line2 = line2.replace(' />','/>')
      data_orginalword2.append(re.findall(p, line2))
      data_orginal_temp2 = line2.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal2.append(data_orginal_temp2)
      data_mask_temp2 = re.sub(p, 'edit', line2)
      if '<edit/>' not in data_mask_temp2:
        print(data_mask_temp)
      data_mask2.append(data_mask_temp2)
      data_replaced_temp2 = data_mask_temp2.replace('<edit/>',row[6])
      data_replaced2.append(data_replaced_temp2)
      data_grades2.append(row[7])
      data_meanGrade2.append([float(row[8])])
      data_editword2.append(row[6])

      data_label.append(int(row[9]))
  return  data_id, data_mask1, data_orginal1, data_replaced1, data_grades1, data_meanGrade1, data_editword1, data_orginalword1, data_mask2, data_orginal2, data_replaced2, data_grades2, data_meanGrade2, data_editword2, data_orginalword2, data_label
data_id, data_mask1, data_orginal1, data_replaced1, data_grades1, data_meanGrade1, data_editword1, data_orginalword1, data_mask2, data_orginal2, data_replaced2, data_grades2, data_meanGrade2, data_editword2, data_orginalword2, data_label = readTask1Data('/content/drive/My Drive/FinalProject/semeval-2020-task-7-dataset/semeval-2020-task-7-dataset/subtask-2/train.csv')
test_id, test_mask1, test_orginal1, test_replaced1, test_grades1, test_meanGrade1, test_editword1, test_orginalword1, test_mask2, test_orginal2, test_replaced2, test_grades2, test_meanGrade2, test_editword2, test_orginalword2, test_label = readTask1Data('/content/drive/My Drive/FinalProject/semeval-2020-task-7-dataset/semeval-2020-task-7-dataset/subtask-2/test.csv')
def splitTrainValid(data_id, data_mask1, data_orginal1, data_replaced1, data_grades1, data_meanGrade1, data_editword1, data_orginalword1, data_mask2, data_orginal2, data_replaced2, data_grades2, data_meanGrade2, data_editword2, data_orginalword2, data_label, valid_num=0.3):
  train_id = list()
  train_mask1 = list()
  train_orginal1 = list()
  train_replaced1 = list()
  train_grades1 = list()
  train_meanGrade1 = list()
  train_editword1 = list()
  train_orginalword1 = list()
  train_mask2 = list()
  train_orginal2 = list()
  train_replaced2 = list()
  train_grades2 = list()
  train_meanGrade2 = list()
  train_editword2 = list()
  train_orginalword2 = list()
  train_label = list()
  val_id = list()
  val_mask1 = list()
  val_orginal1 = list()
  val_replaced1 = list()
  val_grades1 = list()
  val_meanGrade1 = list()
  val_editword1 = list()
  val_orginalword1 = list()
  val_mask2 = list()
  val_orginal2 = list()
  val_replaced2 = list()
  val_grades2 = list()
  val_meanGrade2 = list()
  val_editword2 = list()
  val_orginalword2 = list()
  val_label = list()
  temp_list = list()
  for num in range(len(data_id)):
    temp_list.append([data_id[num], data_mask1[num], data_orginal1[num], data_replaced1[num], data_grades1[num], data_meanGrade1[num], data_editword1[num], data_orginalword1[num], data_mask2[num], data_orginal2[num], data_replaced2[num], data_grades2[num], data_meanGrade2[num], data_editword2[num], data_orginalword2[num], data_label[num]])
  train_set,valid_set = train_test_split(temp_list,test_size = valid_num)
  for (set_id_temp, set_mask1_temp, set_orginal1_temp, set_replaced1_temp, set_grades1_temp, set_meanGrade1_temp, set_editword1_temp, set_orginalword1_temp, set_mask2_temp, set_orginal2_temp, set_replaced2_temp, set_grades2_temp, set_meanGrade2_temp, set_editword2_temp, set_orginalword2_temp, set_label_temp) in train_set:
    train_id.append(set_id_temp)
    train_mask1.append(set_mask1_temp)
    train_orginal1.append(set_orginal1_temp)
    train_replaced1.append(set_replaced1_temp)
    train_grades1.append(set_grades1_temp)
    train_meanGrade1.append(set_meanGrade1_temp)
    train_editword1.append(set_editword1_temp)
    train_orginalword1.append(set_orginalword1_temp)
    train_mask2.append(set_mask2_temp)
    train_orginal2.append(set_orginal2_temp)
    train_replaced2.append(set_replaced2_temp)
    train_grades2.append(set_grades2_temp)
    train_meanGrade2.append(set_meanGrade2_temp)
    train_editword2.append(set_editword2_temp)
    train_orginalword2.append(set_orginalword2_temp)
    train_label.append(set_label_temp)
  for (set_id_temp, set_mask1_temp, set_orginal1_temp, set_replaced1_temp, set_grades1_temp, set_meanGrade1_temp, set_editword1_temp, set_orginalword1_temp, set_mask2_temp, set_orginal2_temp, set_replaced2_temp, set_grades2_temp, set_meanGrade2_temp, set_editword2_temp, set_orginalword2_temp, set_label_temp) in valid_set:
    val_id.append(set_id_temp)
    val_mask1.append(set_mask1_temp)
    val_orginal1.append(set_orginal1_temp)
    val_replaced1.append(set_replaced1_temp)
    val_grades1.append(set_grades1_temp)
    val_meanGrade1.append(set_meanGrade1_temp)
    val_editword1.append(set_editword1_temp)
    val_orginalword1.append(set_orginalword1_temp)
    val_mask2.append(set_mask2_temp)
    val_orginal2.append(set_orginal2_temp)
    val_replaced2.append(set_replaced2_temp)
    val_grades2.append(set_grades2_temp)
    val_meanGrade2.append(set_meanGrade2_temp)
    val_editword2.append(set_editword2_temp)
    val_orginalword2.append(set_orginalword2_temp)
    val_label.append(set_label_temp)
  return train_id, train_mask1, train_orginal1, train_replaced1, train_grades1, train_meanGrade1, train_editword1, train_orginalword1, train_mask2, train_orginal2, train_replaced2, train_grades2, train_meanGrade2, train_editword2, train_orginalword2, train_label, val_id,val_mask1,val_orginal1,val_replaced1,val_grades1,val_meanGrade1,val_editword1,val_orginalword1,val_mask2,val_orginal2,val_replaced2,val_grades2,val_meanGrade2,val_editword2,val_orginalword2,val_label
data_id, data_mask1, data_orginal1, data_replaced1, data_grades1, data_meanGrade1, data_editword1, data_orginalword1, data_mask2, data_orginal2, data_replaced2, data_grades2, data_meanGrade2, data_editword2, data_orginalword2, data_label,val_id,val_mask1,val_orginal1,val_replaced1,val_grades1,val_meanGrade1,val_editword1,val_orginalword1,val_mask2,val_orginal2,val_replaced2,val_grades2,val_meanGrade2,val_editword2,val_orginalword2,val_label = splitTrainValid(data_id, data_mask1, data_orginal1, data_replaced1, data_grades1, data_meanGrade1, data_editword1, data_orginalword1, data_mask2, data_orginal2, data_replaced2, data_grades2, data_meanGrade2, data_editword2, data_orginalword2, data_label)

###With Dev.csv

In [ ]:
import csv
import re
def readTask1Data(data_address):
  data_id = list()
  data_mask1 = list()
  data_orginal1 = list()
  data_replaced1 = list()
  data_grades1 = list()
  data_meanGrade1 = list()
  data_editword1 = list()
  data_orginalword1 = list()
  data_mask2 = list()
  data_orginal2 = list()
  data_replaced2 = list()
  data_grades2 = list()
  data_meanGrade2 = list()
  data_editword2 = list()
  data_orginalword2 = list()
  data_label = list()
  flag = 0
  p = r"(?<=<).+?(?=/>)" 
  with open(data_address, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      data_id.append(row[0])
      line1 = row[1]
      while '  ' in line1:
        line1.replace('  ',' ')
      line1 = line1.replace('/ >','/>')
      line1 = line1.replace('< ','<')
      line1 = line1.replace(' />','/>')
      data_orginalword1.append(re.findall(p, line1))
      data_orginal_temp1 = line1.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal1.append(data_orginal_temp1)
      data_mask_temp1 = re.sub(p, 'edit', line1)
      if '<edit/>' not in data_mask_temp1:
        print(data_mask_temp)
      data_mask1.append(data_mask_temp1)
      data_replaced_temp1 = data_mask_temp1.replace('<edit/>',row[2])
      data_replaced1.append(data_replaced_temp1)
      data_grades1.append(row[3])
      data_meanGrade1.append([float(row[4])])
      data_editword1.append(row[2])


      line2 = row[5]
      while '  ' in line2:
        line2.replace('  ',' ')
      line2 = line2.replace('/ >','/>')
      line2 = line2.replace('< ','<')
      line2 = line2.replace(' />','/>')
      data_orginalword2.append(re.findall(p, line2))
      data_orginal_temp2 = line2.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal2.append(data_orginal_temp2)
      data_mask_temp2 = re.sub(p, 'edit', line2)
      if '<edit/>' not in data_mask_temp2:
        print(data_mask_temp)
      data_mask2.append(data_mask_temp2)
      data_replaced_temp2 = data_mask_temp2.replace('<edit/>',row[6])
      data_replaced2.append(data_replaced_temp2)
      data_grades2.append(row[7])
      data_meanGrade2.append([float(row[8])])
      data_editword2.append(row[6])

      data_label.append(int(row[9]))
  return  data_id, data_mask1, data_orginal1, data_replaced1, data_grades1, data_meanGrade1, data_editword1, data_orginalword1, data_mask2, data_orginal2, data_replaced2, data_grades2, data_meanGrade2, data_editword2, data_orginalword2, data_label
data_id, data_mask1, data_orginal1, data_replaced1, data_grades1, data_meanGrade1, data_editword1, data_orginalword1, data_mask2, data_orginal2, data_replaced2, data_grades2, data_meanGrade2, data_editword2, data_orginalword2, data_label = readTask1Data('/content/drive/My Drive/FinalProject/semeval-2020-task-7-dataset/semeval-2020-task-7-dataset/subtask-2/train.csv')
test_id, test_mask1, test_orginal1, test_replaced1, test_grades1, test_meanGrade1, test_editword1, test_orginalword1, test_mask2, test_orginal2, test_replaced2, test_grades2, test_meanGrade2, test_editword2, test_orginalword2, test_label = readTask1Data('/content/drive/My Drive/FinalProject/semeval-2020-task-7-dataset/semeval-2020-task-7-dataset/subtask-2/test.csv')
dev_id, dev_mask1, dev_orginal1, dev_replaced1, dev_grades1, dev_meanGrade1, dev_editword1, dev_orginalword1, dev_mask2, dev_orginal2, dev_replaced2, dev_grades2, dev_meanGrade2, dev_editword2, dev_orginalword2, dev_label = readTask1Data('/content/drive/My Drive/FinalProject/semeval-2020-task-7-dataset/semeval-2020-task-7-dataset/subtask-2/dev.csv')
data_id = dev_id + data_id
data_mask1 = dev_mask1 + data_mask1
data_orginal1 = dev_orginal1 + data_orginal1
data_replaced1 = dev_replaced1 + data_replaced1
data_grades1 = dev_grades1 + data_grades1
data_meanGrade1 = dev_meanGrade1 + data_meanGrade1
data_editword1 = dev_editword1 + data_editword1
data_orginalword1 = dev_orginalword1 + data_orginalword1
data_mask2 = dev_mask2 + data_mask2
data_orginal2 = dev_orginal2 + data_orginal2
data_replaced2 = dev_replaced2 + data_replaced2
data_grades2 = dev_grades2 + data_grades2
data_meanGrade2 = dev_meanGrade2 + data_meanGrade2
data_editword2 = dev_editword2 + data_editword2
data_orginalword2 = dev_orginalword2 + data_orginalword2
data_label = dev_label + data_label
def splitTrainValid(data_id, data_mask1, data_orginal1, data_replaced1, data_grades1, data_meanGrade1, data_editword1, data_orginalword1, data_mask2, data_orginal2, data_replaced2, data_grades2, data_meanGrade2, data_editword2, data_orginalword2, data_label, valid_num=0.3):
  train_id = list()
  train_mask1 = list()
  train_orginal1 = list()
  train_replaced1 = list()
  train_grades1 = list()
  train_meanGrade1 = list()
  train_editword1 = list()
  train_orginalword1 = list()
  train_mask2 = list()
  train_orginal2 = list()
  train_replaced2 = list()
  train_grades2 = list()
  train_meanGrade2 = list()
  train_editword2 = list()
  train_orginalword2 = list()
  train_label = list()
  val_id = list()
  val_mask1 = list()
  val_orginal1 = list()
  val_replaced1 = list()
  val_grades1 = list()
  val_meanGrade1 = list()
  val_editword1 = list()
  val_orginalword1 = list()
  val_mask2 = list()
  val_orginal2 = list()
  val_replaced2 = list()
  val_grades2 = list()
  val_meanGrade2 = list()
  val_editword2 = list()
  val_orginalword2 = list()
  val_label = list()
  temp_list = list()
  for num in range(len(data_id)):
    temp_list.append([data_id[num], data_mask1[num], data_orginal1[num], data_replaced1[num], data_grades1[num], data_meanGrade1[num], data_editword1[num], data_orginalword1[num], data_mask2[num], data_orginal2[num], data_replaced2[num], data_grades2[num], data_meanGrade2[num], data_editword2[num], data_orginalword2[num], data_label[num]])
  train_set,valid_set = train_test_split(temp_list,test_size = valid_num)
  for (set_id_temp, set_mask1_temp, set_orginal1_temp, set_replaced1_temp, set_grades1_temp, set_meanGrade1_temp, set_editword1_temp, set_orginalword1_temp, set_mask2_temp, set_orginal2_temp, set_replaced2_temp, set_grades2_temp, set_meanGrade2_temp, set_editword2_temp, set_orginalword2_temp, set_label_temp) in train_set:
    train_id.append(set_id_temp)
    train_mask1.append(set_mask1_temp)
    train_orginal1.append(set_orginal1_temp)
    train_replaced1.append(set_replaced1_temp)
    train_grades1.append(set_grades1_temp)
    train_meanGrade1.append(set_meanGrade1_temp)
    train_editword1.append(set_editword1_temp)
    train_orginalword1.append(set_orginalword1_temp)
    train_mask2.append(set_mask2_temp)
    train_orginal2.append(set_orginal2_temp)
    train_replaced2.append(set_replaced2_temp)
    train_grades2.append(set_grades2_temp)
    train_meanGrade2.append(set_meanGrade2_temp)
    train_editword2.append(set_editword2_temp)
    train_orginalword2.append(set_orginalword2_temp)
    train_label.append(set_label_temp)
  for (set_id_temp, set_mask1_temp, set_orginal1_temp, set_replaced1_temp, set_grades1_temp, set_meanGrade1_temp, set_editword1_temp, set_orginalword1_temp, set_mask2_temp, set_orginal2_temp, set_replaced2_temp, set_grades2_temp, set_meanGrade2_temp, set_editword2_temp, set_orginalword2_temp, set_label_temp) in valid_set:
    val_id.append(set_id_temp)
    val_mask1.append(set_mask1_temp)
    val_orginal1.append(set_orginal1_temp)
    val_replaced1.append(set_replaced1_temp)
    val_grades1.append(set_grades1_temp)
    val_meanGrade1.append(set_meanGrade1_temp)
    val_editword1.append(set_editword1_temp)
    val_orginalword1.append(set_orginalword1_temp)
    val_mask2.append(set_mask2_temp)
    val_orginal2.append(set_orginal2_temp)
    val_replaced2.append(set_replaced2_temp)
    val_grades2.append(set_grades2_temp)
    val_meanGrade2.append(set_meanGrade2_temp)
    val_editword2.append(set_editword2_temp)
    val_orginalword2.append(set_orginalword2_temp)
    val_label.append(set_label_temp)
  return train_id, train_mask1, train_orginal1, train_replaced1, train_grades1, train_meanGrade1, train_editword1, train_orginalword1, train_mask2, train_orginal2, train_replaced2, train_grades2, train_meanGrade2, train_editword2, train_orginalword2, train_label, val_id,val_mask1,val_orginal1,val_replaced1,val_grades1,val_meanGrade1,val_editword1,val_orginalword1,val_mask2,val_orginal2,val_replaced2,val_grades2,val_meanGrade2,val_editword2,val_orginalword2,val_label
data_id, data_mask1, data_orginal1, data_replaced1, data_grades1, data_meanGrade1, data_editword1, data_orginalword1, data_mask2, data_orginal2, data_replaced2, data_grades2, data_meanGrade2, data_editword2, data_orginalword2, data_label,val_id,val_mask1,val_orginal1,val_replaced1,val_grades1,val_meanGrade1,val_editword1,val_orginalword1,val_mask2,val_orginal2,val_replaced2,val_grades2,val_meanGrade2,val_editword2,val_orginalword2,val_label = splitTrainValid(data_id, data_mask1, data_orginal1, data_replaced1, data_grades1, data_meanGrade1, data_editword1, data_orginalword1, data_mask2, data_orginal2, data_replaced2, data_grades2, data_meanGrade2, data_editword2, data_orginalword2, data_label)

# Model

##Task1

###OurModel

In [ ]:
configuration.num_labels = 1
class editHeadlineModel(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(editHeadlineModel, self).__init__(conf)
    self.roberta = transformers.RobertaModel.from_pretrained('roberta-base')
    self.drop_out = nn.Dropout(0.1)
    #self.pooling = nn.AdaptiveAvgPool1d(1024)
    self.dense = nn.Linear(768, 768)
    #self.linear = nn.Linear(768, 1)
  def forward(self, input_ids, attention_mask):
    out = self.roberta(input_ids,attention_mask=attention_mask)[0]
    out = out[:,0,:]
    out = self.drop_out(out)
    out = torch.tanh(out)
    out = self.drop_out(out)
    #out = self.linear(out)
    #logits = self.linear(out)
    return out
class wordModel(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(wordModel, self).__init__(conf)
    self.roberta = transformers.RobertaModel.from_pretrained('roberta-base')
    self.drop_out = nn.Dropout(0.1)
    self.dense = nn.Linear(768, 768)
    self.linear = nn.Linear(768, 1)
  def forward(self, input_ids, attention_mask, edit_matrix, edit_total_num):
    batch_size = input_ids.size()[0]
    out = self.roberta(input_ids,attention_mask=attention_mask)[0]
    seq_size = out.size()[1]
    out = torch.bmm(edit_matrix, out)
    #print(out.size(),out.squeeze(dim=1).size())
    out = out/edit_total_num
    out = out.squeeze(dim=1)
    out = self.drop_out(out)
    out = torch.tanh(out)
    out = self.drop_out(out)
    #logits = self.linear(out)
    return out
class twoHeadlineModel_v1(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(twoHeadlineModel_v1, self).__init__(conf)
    self.roberta = transformers.RobertaModel.from_pretrained('roberta-base')
    #self.roberta = transformers.RobertaModel('roberta-base')
    self.dense = nn.Linear(768*4, 768*4)
    self.dropout = nn.Dropout(0.1)
    self.out_proj = nn.Linear(768*4, 1)
    self.linear = nn.Linear(768*4,1)
  def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
    out1 = self.roberta(input_ids1,attention_mask=attention_mask1)[0]
    out2 = self.roberta(input_ids2,attention_mask=attention_mask2)[0]
    out1 = out1[:,0,:]
    out2 = out2[:,0,:]
    merge = [out1, out2, (out1 - out2).abs(), out1 * out2]
    cat = torch.cat(merge, dim=-1)
    out = self.dense(cat)
    #out = self.out_proj(out)
    #logits = self.Linear(out)
    return out
class stakeModel(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(stakeModel, self).__init__(conf)
    self.editHeadlineModel = editHeadlineModel(conf)
    self.wordModel = wordModel(conf)
    self.twoHeadlineModel_v1 = twoHeadlineModel_v1(conf)
    print(1)
    self.dense = nn.Linear(768*6,768*6)
    self.linear =  nn.Linear(768*6,1)
  def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2, edit_matrix, edit_total_num):
    editHeadlineModeloutput = self.editHeadlineModel(input_ids1, attention_mask1)
    wordModeloutput = self.wordModel(input_ids1, attention_mask1, edit_matrix, edit_total_num)
    twoHeadlineModel_v1output = self.twoHeadlineModel_v1(input_ids1, attention_mask1, input_ids2, attention_mask2)
    merge = [editHeadlineModeloutput, wordModeloutput, twoHeadlineModel_v1output]
    cat = torch.cat(merge, dim=-1)
    out = self.dense(cat)
    out = self.linear(out)
    return out
model = stakeModel(configuration)

##RoBERTa

###RoBERTaSingleHeadlineModel

In [ ]:
class editHeadlineModel(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(editHeadlineModel, self).__init__(conf)
    self.roberta = transformers.RobertaModel.from_pretrained('roberta-base')
    self.drop_out = nn.Dropout(0.1)
    #self.pooling = nn.AdaptiveAvgPool1d(1024)
    self.dense = nn.Linear(768, 768)
    self.linear = nn.Linear(768, 1)
  def forward(self, input_ids, attention_mask):
    out = self.roberta(input_ids,attention_mask=attention_mask)[0]
    out = out[:,0,:]
    out = self.drop_out(out)
    out = torch.tanh(out)
    out = self.drop_out(out)
    out = self.linear(out)
    return out
model = editHeadlineModel(configuration)
model_name = 'editHeadlineModel'

###RoBERTaEditedWordModel

In [ ]:
class wordModel(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(wordModel, self).__init__(conf)
    self.roberta = transformers.RobertaModel.from_pretrained('roberta-base')
    self.drop_out = nn.Dropout(0.1)
    self.dense = nn.Linear(768, 768)
    self.linear = nn.Linear(768, 1)
  def forward(self, input_ids, attention_mask, edit_matrix, edit_total_num):
    batch_size = input_ids.size()[0]
    out = self.roberta(input_ids,attention_mask=attention_mask)[0]
    seq_size = out.size()[1]
    #print(edit_matrix.size(), out.size())
    out = torch.bmm(edit_matrix, out)
    #print(out.size(),out.squeeze(dim=1).size())
    out = out/edit_total_num
    out = out.squeeze(dim=1)
    out = self.drop_out(out)
    out = torch.tanh(out)
    out = self.drop_out(out)
    logits = self.linear(out)
    return logits
model = wordModel(configuration)
model_name = 'wordModel'

###RoBERTaTwoHeadlineModel

In [ ]:
class twoHeadlineModel_v1(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(twoHeadlineModel_v1, self).__init__(conf)
    self.roberta = transformers.RobertaModel.from_pretrained('roberta-base')
    #self.roberta = transformers.RobertaModel('roberta-base')
    self.dense = nn.Linear(768*4, 768*4)
    self.dropout = nn.Dropout(0.1)
    self.linear = nn.Linear(768*4,1)
  def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
    out1 = self.roberta(input_ids1,attention_mask=attention_mask1)[0]
    out2 = self.roberta(input_ids2,attention_mask=attention_mask2)[0]
    out1 = out1[:,0,:]
    out2 = out2[:,0,:]
    merge = [out1, out2, (out1 - out2).abs(), out1 * out2]
    cat = torch.cat(merge, dim=-1)
    out = self.dense(cat)
    logits = self.linear(out)
    return logits
model = twoHeadlineModel_v1(configuration)
model_name = 'twoHeadlineModel_v1'

###RoBERTaSequenceClassification

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', return_dict=True, num_labels=1)
model_name = 'OrginalRoBERTaSequenceClassification'

##BERT

###Configuration&Tokenizer

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
configuration = RobertaConfig()
tokenizer = BertTokenizer.from_pretrained("bert-base-cased", do_lower_case=True)

###BERTSingleHeadlineModel

In [ ]:
class editHeadlineModel(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(editHeadlineModel, self).__init__(conf)
    self.bert = transformers.BertModel.from_pretrained('bert-base-cased')
    self.drop_out = nn.Dropout(0.1)
    #self.pooling = nn.AdaptiveAvgPool1d(1024)
    self.dense = nn.Linear(768, 768)
    self.linear = nn.Linear(768, 1)
  def forward(self, input_ids, attention_mask):
    out = self.bert(input_ids,attention_mask=attention_mask)[0]
    out = out[:,0,:]
    out = self.drop_out(out)
    out = torch.tanh(out)
    out = self.drop_out(out)
    out = self.linear(out)
    return out
model = editHeadlineModel(configuration)
model_name = 'editHeadlineModel'

###BERTEditedWordModel

In [ ]:
class wordModel(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(wordModel, self).__init__(conf)
    self.bert = transformers.BertModel.from_pretrained('bert-base-cased')
    self.drop_out = nn.Dropout(0.1)
    self.dense = nn.Linear(768, 768)
    self.linear = nn.Linear(768, 1)
  def forward(self, input_ids, attention_mask, edit_matrix, edit_total_num):
    batch_size = input_ids.size()[0]
    out = self.bert(input_ids,attention_mask=attention_mask)[0]
    seq_size = out.size()[1]
    #print(edit_matrix.size(), out.size())
    out = torch.bmm(edit_matrix, out)
    #print(out.size(),out.squeeze(dim=1).size())
    out = out/edit_total_num
    out = out.squeeze(dim=1)
    out = self.drop_out(out)
    out = torch.tanh(out)
    out = self.drop_out(out)
    logits = self.linear(out)
    return logits
model = wordModel(configuration)
model_name = 'wordModel'

###BERTTwoHeadlineModel

In [ ]:
class twoHeadlineModel_v1(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(twoHeadlineModel_v1, self).__init__(conf)
    self.bert = transformers.BertModel.from_pretrained('bert-base-cased')
    #self.roberta = transformers.RobertaModel('roberta-base')
    self.dense = nn.Linear(768*4, 768*4)
    self.dropout = nn.Dropout(0.1)
    self.linear = nn.Linear(768*4,1)
  def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
    out1 = self.bert(input_ids1,attention_mask=attention_mask1)[0]
    out2 = self.bert(input_ids2,attention_mask=attention_mask2)[0]
    out1 = out1[:,0,:]
    out2 = out2[:,0,:]
    merge = [out1, out2, (out1 - out2).abs(), out1 * out2]
    cat = torch.cat(merge, dim=-1)
    out = self.dense(cat)
    logits = self.linear(out)
    return logits
model = twoHeadlineModel_v1(configuration)
model_name = 'twoHeadlineModel_v1'

###BERTSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-cased', return_dict=True, num_labels=1)
model_name = 'OrginalRoBERTaSequenceClassification'

##LSTM

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import gensim
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from math import sqrt
import string
import re
import math
import codecs
import random
import nltk
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from collections import defaultdict
from scipy.spatial.distance import euclidean, cosine
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm 
from nltk import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words_list = stopwords.words('english')
from nltk.tag import pos_tag
nltk.download('averaged_perceptron_tagger')

###LSTMSingleHeadlineModel

In [ ]:
class LSTMSingleHeadlineModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, out_size, max_length, n_layer = 1, dropout = 0.3, pool_feature_num = 200):
        super(LSTMSingleHeadlineModel, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.max_length = max_length
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0).cuda()
        self.n_layer = n_layer
        print('Initialization LSTM Model')
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, dropout=dropout, num_layers=self.n_layer, bidirectional=False).cuda()
        self.pooling = nn.AdaptiveMaxPool1d(pool_feature_num).cuda()
        self.out = nn.Linear(pool_feature_num, out_size).cuda()
    def forward(self, X):
        print(X.size())
        input = self.embedding(X)
        input = input.permute(1, 0, 2)
        batch_size = input.size()[1]
        hidden_state = Variable(
            torch.randn(self.n_layer, batch_size, self.hidden_dim)).cuda()
        cell_state = Variable(
            torch.randn(self.n_layer, batch_size, self.hidden_dim)).cuda()
        output, (final_hidden_state, final_cell_state) = self.rnn(input, (hidden_state, cell_state))
        #output = output.permute(1, 0, 2).squeeze(1)
        #output = output.reshape(batch_size, self.hidden_dim * self.max_length)
        #print(final_hidden_state.size())
        #final_hidden_state = final_hidden_state.permute(1, 0, 2).squeeze(1)
        output = output.permute(1, 0, 2)
        output = output.reshape(1, batch_size, self.hidden_dim * X.size()[1])
        pool = self.pooling(output)
        pool = pool.permute(1, 0, 2)
        pool = pool.squeeze(dim=1)
        output = self.out(pool)

        return  output # model : [batch_size, num_classes], attention : [batch_size, n_step]

###LSTMEditedWordModel

In [ ]:
class LSTMEditedWordModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, out_size, max_length, n_layer = 1, dropout = 0.3):
        super(LSTMEditedWordModel, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.max_length = max_length
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0).cuda()
        self.n_layer = n_layer
        print('Initialization LSTM Model')
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, dropout=dropout, num_layers=self.n_layer, bidirectional=False).cuda()
        self.out = nn.Linear(hidden_dim, out_size).cuda()
    def forward(self, X, edit_matrix, edit_total_num):
        print(X.size())
        input = self.embedding(X)
        input = input.permute(1, 0, 2)
        batch_size = input.size()[1]
        hidden_state = Variable(
            torch.randn(self.n_layer, batch_size, self.hidden_dim)).cuda()
        cell_state = Variable(
            torch.randn(self.n_layer, batch_size, self.hidden_dim)).cuda()
        output, (final_hidden_state, final_cell_state) = self.rnn(input, (hidden_state, cell_state))
        #output = output.permute(1, 0, 2).squeeze(1)
        #output = output.reshape(batch_size, self.hidden_dim * self.max_length)
        #print(final_hidden_state.size())
        
        edit_matrix = edit_matrix.cuda()
        out = output.permute(1, 0, 2).squeeze(1)
        print(output.size(), edit_matrix.size())
        out = torch.bmm(edit_matrix, out).cuda()
        out = out/edit_total_num.cuda()
        out = out.squeeze(dim=1)
        out = self.out(out)
        return out

        #return  output # model : [batch_size, num_classes], attention : [batch_size, n_step]

###LSTMTwoHeadlineModel

In [ ]:
class LSTMTwoHeadlineModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, out_size, max_length, n_layer = 1, dropout = 0.3):
        super(LSTMTwoHeadlineModel, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.max_length = max_length
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0).cuda()
        self.n_layer = n_layer
        print('Initialization LSTM Model')
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, dropout=dropout, num_layers=self.n_layer, bidirectional=False).cuda()
        self.dense = nn.Linear(hidden_dim*4, hidden_dim*4).cuda()
        self.dropout = nn.Dropout(0.1).cuda()
        self.linear = nn.Linear(hidden_dim*4,1).cuda()
    def forward(self, X1, X2):
        input1 = self.embedding(X1)
        input2 = self.embedding(X2)
        input1 = input1.permute(1, 0, 2)
        input2 = input2.permute(1, 0, 2)
        batch_size1 = input1.size()[1]
        batch_size2 = input2.size()[1]
        hidden_state1 = Variable(
            torch.randn(self.n_layer, batch_size1, self.hidden_dim)).cuda()
        hidden_state2 = Variable(
            torch.randn(self.n_layer, batch_size2, self.hidden_dim)).cuda()
        cell_state1 = Variable(
            torch.randn(self.n_layer, batch_size1, self.hidden_dim)).cuda()
        cell_state2 = Variable(
            torch.randn(self.n_layer, batch_size2, self.hidden_dim)).cuda()
        output1, (final_hidden_state1, final_cell_state1) = self.rnn(input1, (hidden_state1, cell_state1))
        output2, (final_hidden_state2, final_cell_state2) = self.rnn(input2, (hidden_state2, cell_state2))
        #output = output.permute(1, 0, 2).squeeze(1)
        #output = output.reshape(batch_size, self.hidden_dim * self.max_length)
        #print(final_hidden_state.size())
        #final_hidden_state = final_hidden_state.permute(1, 0, 2).squeeze(1)
        #output = self.out(final_hidden_state)
        output1 = final_hidden_state1.permute(1, 0, 2).squeeze(1)
        output2 = final_hidden_state2.permute(1, 0, 2).squeeze(1)
        print(output1.size(),final_hidden_state1.size(),final_cell_state1.size(),output2.size(),final_hidden_state2.size(),final_cell_state2.size())
        merge = [output1, output2, (output1 - output2).abs(), output1 * output2]
        cat = torch.cat(merge, dim=-1)
        out = self.dense(cat)
        logits = self.linear(out)

        return  logits # model : [batch_size, num_classes], attention : [batch_size, n_step]

###LSTMSequenceClassification

In [ ]:
class LSTMAtt(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, out_size, max_length, n_layer = 1, dropout = 0.3):
        super(LSTMAtt, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.max_length = max_length
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0).cuda()
        self.n_layer = n_layer
        print('Initialization LSTM Model')
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, dropout=dropout, num_layers=self.n_layer, bidirectional=False).cuda()
        self.out = nn.Linear(hidden_dim, out_size).cuda()
    def forward(self, X):
        print(X.size())
        input = self.embedding(X)
        input = input.permute(1, 0, 2)
        batch_size = input.size()[1]
        hidden_state = Variable(
            torch.randn(self.n_layer, batch_size, self.hidden_dim)).cuda()
        cell_state = Variable(
            torch.randn(self.n_layer, batch_size, self.hidden_dim)).cuda()
        output, (final_hidden_state, final_cell_state) = self.rnn(input, (hidden_state, cell_state))
        #output = output.permute(1, 0, 2).squeeze(1)
        #output = output.reshape(batch_size, self.hidden_dim * self.max_length)
        #print(final_hidden_state.size())
        final_hidden_state = final_hidden_state.permute(1, 0, 2).squeeze(1)
        output = self.out(final_hidden_state)

        return  output # model : [batch_size, num_classes], attention : [batch_size, n_step]

#train

##RoBERTa

###Hyperparameter Setting

In [ ]:
lr = 3e-5
max_grad_norm = 1.0
totalEpoch = 1
eps = 1e-8
batchSize = 16
num_train_steps = totalEpoch
WU = 0.2
num_training_steps = ((len(data_id)/batchSize)+1) * totalEpoch
num_warmup_steps = int(num_training_steps * WU)
#scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)

###Optimizer&Scheduler&Loss

In [ ]:
optimizer = AdamW(model.parameters(), lr = lr, eps = eps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_train_steps, num_training_steps=num_training_steps)
loss_fn = nn.MSELoss()
#loss_fn = nn.CrossEntropyLoss()##Task2

###SingleHeadlineModel

In [ ]:
device = torch.device("cuda")
model.to(device)
for epoch in range(totalEpoch):
  batchStart = 0
  train_loss = 0
  optimizer.zero_grad()
  while batchStart <= len(data_meanGrade):
    batchEnd = batchStart + batchSize
    if batchEnd > len(data_meanGrade):
      batchEnd = len(data_meanGrade)
    encoding = tokenizer(data_replaced[batchStart:batchEnd], return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    labels = torch.tensor(data_meanGrade[batchStart:batchEnd]).to(device)
    model.train()
    outputs = model(input_ids, attention_mask=attention_mask)
    #outputs = model(input_ids, attention_mask=attention_mask)
    #loss = torch.sqrt(loss_fn(outputs,labels))
    loss = torch.sqrt(((outputs - labels)**2).mean())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    batchStart = batchStart + batchSize
  with torch.no_grad():
    encoding = tokenizer(val_replaced, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)
    #outputs = model(input_ids, attention_mask=attention_mask)
    labels = torch.tensor(val_meanGrade).to(device)
    loss = torch.sqrt(loss_fn(outputs,labels))
    valid_loss = loss.item()
    print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

###EditedWordModel

In [ ]:
def getEditMatrix(tokenizer, encoding_sentence, encoding_index, replace_matrix, replace_words):
  edit_matrix = torch.zeros(encoding_sentence.size()[0], 1, encoding_sentence.size()[1])
  edit_total_num = torch.zeros(encoding_sentence.size()[0], 1, 1)
  for batch_ in range(len(replace_words)):
    token_list = tokenizer.convert_ids_to_tokens(encoding_index[batch_][1:-1])
    token_str= " ".join(token_list)
    token_str = token_str.split('Ġ')
    token_dict = dict()
    for temp in token_str:
      token_dict[temp.replace(' ','')] = ('Ġ' + temp).split(' ')
    for key in token_dict:
      if replace_words[batch_].lower() in key.lower() and float(len(replace_words[batch_])/len(key)) > 0.5:
        for tok_num in range(len(token_dict[key])):
          if len(token_dict[key]) == 2 and token_dict[key][0].replace('Ġ','') in token_list:
            edit_matrix[batch_][0][encoding_index[batch_].index(tokenizer.convert_tokens_to_ids(token_dict[key][0].replace('Ġ','')))] = 1
            continue
          if token_dict[key][tok_num] == '':
            continue
          edit_token_ids = tokenizer.convert_tokens_to_ids(token_dict[key][tok_num])
          edit_matrix[batch_][0][encoding_index[batch_].index(tokenizer.convert_tokens_to_ids(token_dict[key][tok_num]))] = 1
  for batch_ in range(edit_matrix.size()[0]):
    edit_total_num[batch_][0][0] = edit_matrix[batch_].sum()
  return edit_matrix, edit_total_num
device = torch.device("cuda")
model.to(device)
for epoch in range(totalEpoch):
  batchStart = 0
  train_loss = 0
  optimizer.zero_grad()
  while batchStart <= len(data_meanGrade):
    batchEnd = batchStart + batchSize
    if batchEnd > len(data_meanGrade):
      batchEnd = len(data_meanGrade)
    encoding = tokenizer(data_replaced[batchStart:batchEnd], return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    labels = torch.tensor(data_meanGrade[batchStart:batchEnd]).to(device)
    encoding_index = tokenizer(data_replaced[batchStart:batchEnd], add_special_tokens=data_editword)['input_ids']
    replace_words = data_editword[batchStart:batchEnd]
    replace_matrix = tokenizer(replace_words, return_tensors='pt', padding=True, truncation=True, add_special_tokens=data_editword)['input_ids']
    edit_matrix, edit_total_num = getEditMatrix(tokenizer,encoding['input_ids'],encoding_index,replace_matrix,replace_words)
    model.train()
    #outputs = model(input_ids, attention_mask=attention_mask)[0]
    outputs = model(input_ids, attention_mask=attention_mask,edit_matrix=edit_matrix.to(device), edit_total_num=edit_total_num.to(device))
    #loss = torch.sqrt(loss_fn(outputs,labels))
    loss = torch.sqrt(((outputs - labels)**2).mean())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    batchStart = batchStart + batchSize
    
  with torch.no_grad():
    encoding = tokenizer(val_replaced, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    #outputs = model(input_ids, attention_mask=attention_mask)[0]
    encoding_index = tokenizer(val_replaced, add_special_tokens=val_editword)['input_ids']
    replace_words = val_editword
    replace_matrix = tokenizer(replace_words, return_tensors='pt', padding=True, truncation=True, add_special_tokens=replace_words)['input_ids']
    edit_matrix, edit_total_num = getEditMatrix(tokenizer,encoding['input_ids'],encoding_index,replace_matrix,replace_words)
    outputs = model(input_ids, attention_mask=attention_mask,edit_matrix=edit_matrix.to(device), edit_total_num=edit_total_num.to(device))
    labels = torch.tensor(val_meanGrade).to(device)
    #loss = torch.sqrt(loss_fn(outputs,labels))
    loss = torch.sqrt(((outputs - labels)**2).mean())
    valid_loss = loss.item()
    print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

###TwoHeadlineModel

In [ ]:
device = torch.device("cuda")
model.to(device)
for epoch in range(totalEpoch):
  batchStart = 0
  train_loss = 0
  optimizer.zero_grad()
  while batchStart <= len(data_meanGrade):
    batchEnd = batchStart + batchSize
    if batchEnd > len(data_meanGrade):
      batchEnd = len(data_meanGrade)
    old_title = data_orginal[batchStart:batchEnd]
    new_title = data_replaced[batchStart:batchEnd]
    encoding_old = tokenizer(old_title, return_tensors='pt', padding=True, truncation=True)
    encoding_new = tokenizer(new_title, return_tensors='pt', padding=True, truncation=True)
    input_ids_old = encoding_old['input_ids'].to(device)
    input_ids_new = encoding_new['input_ids'].to(device)
    attention_mask_old = encoding_old['attention_mask'].to(device)
    attention_mask_new = encoding_new['attention_mask'].to(device)
    labels = torch.tensor(data_meanGrade[batchStart:batchEnd]).to(device)
    model.train()
    #outputs = model(input_ids_new, input_ids_old, attention_mask_new, attention_mask_old)[0]
    outputs = model(input_ids_new, attention_mask_new, input_ids_old, attention_mask_old)
    #loss = torch.sqrt(loss_fn(outputs,labels))
    loss = torch.sqrt(((outputs - labels)**2).mean())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    batchStart = batchStart + batchSize
    
  with torch.no_grad():
    new_title = val_replaced
    old_title = val_orginal
    encoding_old = tokenizer(old_title, return_tensors='pt', padding=True, truncation=True)
    encoding_new = tokenizer(new_title, return_tensors='pt', padding=True, truncation=True)
    input_ids_old = encoding_old['input_ids'].to(device)
    input_ids_new = encoding_new['input_ids'].to(device)
    attention_mask_old = encoding_old['attention_mask'].to(device)
    attention_mask_new = encoding_new['attention_mask'].to(device)
    #outputs = model(input_ids_new, input_ids_old, attention_mask_new, attention_mask_old)[0]
    outputs = model(input_ids_new, attention_mask_new, input_ids_old, attention_mask_old)
    labels = torch.tensor(val_meanGrade).to(device)
    loss = torch.sqrt(loss_fn(outputs,labels))
    valid_loss = loss.item()
    print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

In [ ]:
outputs.size()

###SequenceClassification

In [ ]:
device = torch.device("cuda")
model.to(device)
for epoch in range(totalEpoch):
  batchStart = 0
  train_loss = 0
  optimizer.zero_grad()
  while batchStart <= len(data_meanGrade):
    batchEnd = batchStart + batchSize
    if batchEnd > len(data_meanGrade):
      batchEnd = len(data_meanGrade)
    input_title = data_replaced[batchStart:batchEnd]
    input_word = data_editword[batchStart:batchEnd]
    input_data = [[input_title[num_],input_word[num_]] for num_ in range(len(input_title))]
    encoding = tokenizer(input_data, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    labels = torch.tensor(data_meanGrade[batchStart:batchEnd]).to(device)
    model.train()
    outputs = model(input_ids, attention_mask=attention_mask)[0]
    #outputs = model(input_ids, attention_mask=attention_mask)
    loss = torch.sqrt(loss_fn(outputs,labels))
    #loss = 2*torch.sqrt(((outputs - labels)**2).mean())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    batchStart = batchStart + batchSize
    
  with torch.no_grad():
    input_title = val_replaced
    input_word = val_editword
    input_data = [[input_title[num_],input_word[num_]] for num_ in range(len(input_title))]
    encoding = tokenizer(input_data, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)[0]
    #outputs = model(input_ids, attention_mask=attention_mask)
    labels = torch.tensor(val_meanGrade).to(device)
    loss = torch.sqrt(loss_fn(outputs,labels))
    valid_loss = loss.item()
    print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

##BERT

###Hyperparameter Setting

In [ ]:
lr = 3e-5
max_grad_norm = 1.0
totalEpoch = 1
eps = 1e-8
batchSize = 16
num_train_steps = totalEpoch
WU = 0.2
num_training_steps = ((len(data_id)/batchSize)+1) * totalEpoch
num_warmup_steps = int(num_training_steps * WU)
#scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)

###Optimizer&Scheduler&Loss

In [ ]:
optimizer = AdamW(model.parameters(), lr = lr, eps = eps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_train_steps, num_training_steps=num_training_steps)
loss_fn = nn.MSELoss()
#loss_fn = nn.CrossEntropyLoss()##Task2

###SingleHeadlineModel

In [ ]:
device = torch.device("cuda")
model.to(device)
for epoch in range(totalEpoch):
  batchStart = 0
  train_loss = 0
  optimizer.zero_grad()
  while batchStart <= len(data_meanGrade):
    batchEnd = batchStart + batchSize
    if batchEnd > len(data_meanGrade):
      batchEnd = len(data_meanGrade)
    encoding = tokenizer(data_replaced[batchStart:batchEnd], return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    labels = torch.tensor(data_meanGrade[batchStart:batchEnd]).to(device)
    model.train()
    outputs = model(input_ids, attention_mask=attention_mask)
    #outputs = model(input_ids, attention_mask=attention_mask)
    #loss = torch.sqrt(loss_fn(outputs,labels))
    loss = torch.sqrt(((outputs - labels)**2).mean())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    batchStart = batchStart + batchSize
  with torch.no_grad():
    encoding = tokenizer(val_replaced, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)
    #outputs = model(input_ids, attention_mask=attention_mask)
    labels = torch.tensor(val_meanGrade).to(device)
    loss = torch.sqrt(loss_fn(outputs,labels))
    valid_loss = loss.item()
    print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

In [ ]:
input_ids.size()

In [ ]:
attention_mask.size()

###EditedWordModel

In [ ]:
def getEditMatrix(tokenizer, encoding_sentence, encoding_index, replace_matrix, replace_words):
  edit_matrix = torch.zeros(encoding_sentence.size()[0], 1, encoding_sentence.size()[1])
  edit_total_num = torch.zeros(encoding_sentence.size()[0], 1, 1)
  for batch_ in range(len(replace_words)):
    token_list = tokenizer.convert_ids_to_tokens(encoding_index[batch_][1:-1])
    token_str= " ".join(token_list)
    token_str = token_str.split('Ġ')
    token_dict = dict()
    for temp in token_str:
      token_dict[temp.replace(' ','')] = ('Ġ' + temp).split(' ')
    for key in token_dict:
      if replace_words[batch_].lower() in key.lower() and float(len(replace_words[batch_])/len(key)) > 0.5:
        for tok_num in range(len(token_dict[key])):
          if len(token_dict[key]) == 2 and token_dict[key][0].replace('Ġ','') in token_list:
            edit_matrix[batch_][0][encoding_index[batch_].index(tokenizer.convert_tokens_to_ids(token_dict[key][0].replace('Ġ','')))] = 1
            continue
          if token_dict[key][tok_num] == '':
            continue
          edit_token_ids = tokenizer.convert_tokens_to_ids(token_dict[key][tok_num])
          edit_matrix[batch_][0][encoding_index[batch_].index(tokenizer.convert_tokens_to_ids(token_dict[key][tok_num]))] = 1
  for batch_ in range(edit_matrix.size()[0]):
    edit_total_num[batch_][0][0] = edit_matrix[batch_].sum()
  return edit_matrix, edit_total_num
device = torch.device("cuda")
model.to(device)
for epoch in range(totalEpoch):
  batchStart = 0
  train_loss = 0
  optimizer.zero_grad()
  while batchStart <= len(data_meanGrade):
    batchEnd = batchStart + batchSize
    if batchEnd > len(data_meanGrade):
      batchEnd = len(data_meanGrade)
    encoding = tokenizer(data_replaced[batchStart:batchEnd], return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    labels = torch.tensor(data_meanGrade[batchStart:batchEnd]).to(device)
    encoding_index = tokenizer(data_replaced[batchStart:batchEnd], add_special_tokens=data_editword)['input_ids']
    replace_words = data_editword[batchStart:batchEnd]
    replace_matrix = tokenizer(replace_words, return_tensors='pt', padding=True, truncation=True, add_special_tokens=data_editword)['input_ids']
    edit_matrix, edit_total_num = getEditMatrix(tokenizer,encoding['input_ids'],encoding_index,replace_matrix,replace_words)
    model.train()
    #outputs = model(input_ids, attention_mask=attention_mask)[0]
    outputs = model(input_ids, attention_mask=attention_mask,edit_matrix=edit_matrix.to(device), edit_total_num=edit_total_num.to(device))
    #loss = torch.sqrt(loss_fn(outputs,labels))
    loss = torch.sqrt(((outputs - labels)**2).mean())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    batchStart = batchStart + batchSize
    
  with torch.no_grad():
    encoding = tokenizer(val_replaced, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    #outputs = model(input_ids, attention_mask=attention_mask)[0]
    encoding_index = tokenizer(val_replaced, add_special_tokens=val_editword)['input_ids']
    replace_words = val_editword
    replace_matrix = tokenizer(replace_words, return_tensors='pt', padding=True, truncation=True, add_special_tokens=replace_words)['input_ids']
    edit_matrix, edit_total_num = getEditMatrix(tokenizer,encoding['input_ids'],encoding_index,replace_matrix,replace_words)
    outputs = model(input_ids, attention_mask=attention_mask,edit_matrix=edit_matrix.to(device), edit_total_num=edit_total_num.to(device))
    labels = torch.tensor(val_meanGrade).to(device)
    #loss = torch.sqrt(loss_fn(outputs,labels))
    loss = torch.sqrt(((outputs - labels)**2).mean())
    valid_loss = loss.item()
    print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

###TwoHeadlineModel

In [ ]:
device = torch.device("cuda")
model.to(device)
for epoch in range(totalEpoch):
  batchStart = 0
  train_loss = 0
  optimizer.zero_grad()
  while batchStart <= len(data_meanGrade):
    batchEnd = batchStart + batchSize
    if batchEnd > len(data_meanGrade):
      batchEnd = len(data_meanGrade)
    old_title = data_orginal[batchStart:batchEnd]
    new_title = data_replaced[batchStart:batchEnd]
    encoding_old = tokenizer(old_title, return_tensors='pt', padding=True, truncation=True)
    encoding_new = tokenizer(new_title, return_tensors='pt', padding=True, truncation=True)
    input_ids_old = encoding_old['input_ids'].to(device)
    input_ids_new = encoding_new['input_ids'].to(device)
    attention_mask_old = encoding_old['attention_mask'].to(device)
    attention_mask_new = encoding_new['attention_mask'].to(device)
    labels = torch.tensor(data_meanGrade[batchStart:batchEnd]).to(device)
    model.train()
    #outputs = model(input_ids_new, input_ids_old, attention_mask_new, attention_mask_old)[0]
    outputs = model(input_ids_new, attention_mask_new, input_ids_old, attention_mask_old)
    #loss = torch.sqrt(loss_fn(outputs,labels))
    loss = torch.sqrt(((outputs - labels)**2).mean())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    batchStart = batchStart + batchSize
    
  with torch.no_grad():
    new_title = val_replaced
    old_title = val_orginal
    encoding_old = tokenizer(old_title, return_tensors='pt', padding=True, truncation=True)
    encoding_new = tokenizer(new_title, return_tensors='pt', padding=True, truncation=True)
    input_ids_old = encoding_old['input_ids'].to(device)
    input_ids_new = encoding_new['input_ids'].to(device)
    attention_mask_old = encoding_old['attention_mask'].to(device)
    attention_mask_new = encoding_new['attention_mask'].to(device)
    #outputs = model(input_ids_new, input_ids_old, attention_mask_new, attention_mask_old)[0]
    outputs = model(input_ids_new, attention_mask_new, input_ids_old, attention_mask_old)
    labels = torch.tensor(val_meanGrade).to(device)
    loss = torch.sqrt(loss_fn(outputs,labels))
    valid_loss = loss.item()
    print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

In [ ]:
outputs.size()

###SequenceClassification

In [ ]:
device = torch.device("cuda")
model.to(device)
for epoch in range(totalEpoch):
  batchStart = 0
  train_loss = 0
  optimizer.zero_grad()
  while batchStart <= len(data_meanGrade):
    batchEnd = batchStart + batchSize
    if batchEnd > len(data_meanGrade):
      batchEnd = len(data_meanGrade)
    input_title = data_replaced[batchStart:batchEnd]
    input_word = data_editword[batchStart:batchEnd]
    input_data = [[input_title[num_],input_word[num_]] for num_ in range(len(input_title))]
    encoding = tokenizer(input_data, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    labels = torch.tensor(data_meanGrade[batchStart:batchEnd]).to(device)
    model.train()
    outputs = model(input_ids, attention_mask=attention_mask)[0]
    #outputs = model(input_ids, attention_mask=attention_mask)
    loss = torch.sqrt(loss_fn(outputs,labels))
    #loss = 2*torch.sqrt(((outputs - labels)**2).mean())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    batchStart = batchStart + batchSize
    
  with torch.no_grad():
    input_title = val_replaced
    input_word = val_editword
    input_data = [[input_title[num_],input_word[num_]] for num_ in range(len(input_title))]
    encoding = tokenizer(input_data, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)[0]
    #outputs = model(input_ids, attention_mask=attention_mask)
    labels = torch.tensor(val_meanGrade).to(device)
    loss = torch.sqrt(loss_fn(outputs,labels))
    valid_loss = loss.item()
    print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

##LSTM

###Original Setting


In [ ]:
EMBEDDING_DIM = 300

SCALE_EMBS = 1

hidden_dim = 20

out_size = 1

def fix_seed(seed=234):
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)
fix_seed()

###PreProcessMethod

In [ ]:
def pre_process(sentence):
  sentence_temp = sentence.lower()
  sentence_tokenize = word_tokenize(sentence)
  sentence_tokenize = nltk.pos_tag(sentence_tokenize)
  pos_sentence = []
  for token in sentence_tokenize: 
    if token[1] == "NNP" or token[1] == "NN":
      pos_sentence.append(token[0].capitalize())
    else:
      pos_sentence.append(token[0])
  stop_word_removed_sentence = []
  for token in pos_sentence:
    if token not in stop_words_list:
      stop_word_removed_sentence.append(token)
  return_sentence = " ".join(stop_word_removed_sentence)
  return return_sentence

def getCorpus(data_address):
  data_replaced = list()
  data_id = list()
  labels = list()
  file = open(data_address,'r',encoding='utf-8')
  file_read = file.readlines()
  flag = 0
  for line in file_read:
    if flag == 0:
      flag = 1
      continue
    line = line.replace('\n','').replace('\r','')
    line = line.split(',')
    if len(line) > 5:
      line_temp = list()
      line_temp.append(line[0])
      line_content = ','.join(line[1:-3])
      line_content_temp = line_content.split(' ')
      for num in range(len(line_content_temp)):
        if '<' in line_content_temp[num] and '/>' in line_content_temp[num]:
          line_content_temp[num] = line[-3]
      line_content = ' '.join(line_content_temp)
      line_content = pre_process(line_content)
      line_temp.append(line_content)
      line_temp.append(line[-3])
      line_temp.append(line[-2])
      line_temp.append(line[-1])
      line = line_temp
    else:
      line_content_temp = line[1].split(' ')
      for num in range(len(line_content_temp)):
        if '<' in line_content_temp[num] and '/>' in line_content_temp[num]:
          line_content_temp[num] = line[-3]
      line_content = ' '.join(line_content_temp)
      line[1] = line_content
    line_temp = line[1].replace(' ','')
    if '<' in line_temp and '/>' in line_temp:
      p = r"(?<=<).+?(?=/>)" 
      line[1] = line[1].replace('/ >','/>')
      line[1] = re.sub(p, line[2], line[1])
      line[1] = line[1].replace('<','').replace('/>','')
      line[1] = pre_process(line[1])
    #orginal_data.append(line)
    data_id.append(line[0])
    data_replaced.append(line[1])
    labels.append(float(line[4]))
  file.close()
  return data_id, data_replaced, labels
def average_error(output, target):
  output = torch.round(torch.sigmoid(output))
  err_num = 0
  err_mean = 0.0
  err_total = 0.0
  for num in range(len(output)):
    err_total = err_total + abs(output[num].item() - target[num])
  err_mean = float(err_total/len(output))
  return err_mean
def get_tokenized_corpus(corpus):
  tokenized_corpus = []
  for sent in corpus:
    tokenised_sentence = []
    for token in sent.split(' '):
      tokenised_sentence.append(token)
    tokenized_corpus.append(tokenised_sentence)
  return tokenized_corpus
def get_word2idx(tokenized_corpus):
  vocabulary = []
  for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)
  
  word2idx = {w: idx+1 for (idx, w) in enumerate(vocabulary)}
  # we reserve the 0 index for the padding token
  word2idx['<pad>'] = 0

In [ ]:
def getTokenizedCorpus(corpus):
  tokenized_corpus = []
  for sentence in corpus:
    tokenized_sentence = []
    for token in sentence.split(' '): 
      tokenized_sentence.append(token)
    tokenized_corpus.append(tokenized_sentence)
  return tokenized_corpus

###GetData

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
# Unzip the file: 4 different embedding sizes are provided
!unzip glove.6B.zip
# Check the file format
!head -n10 glove.6B.50d.txt

In [ ]:
train_id, train, train_labels = getCorpus('/content/drive/My Drive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test, test_labels = getCorpus('/content/drive/My Drive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
valid_id, valid, valid_labels = getCorpus('/content/drive/My Drive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')

train = getTokenizedCorpus(train)
test = getTokenizedCorpus(test)
valid = getTokenizedCorpus(valid)

###WordVector

In [ ]:
def getWord2idx(tokenized_corpus):
  vocabulary = []
  for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)
  word2idx = {w: idx+1 for (idx, w) in enumerate(vocabulary)}
  word2idx['<pad>'] = 0
  return word2idx
def encodeData(tokenized_corpus, word2idx):
  enocded_corpus = list()
  for sentence in tokenized_corpus:
    encoded_sentence = list()
    for token in sentence:
      encoded_sentence.append(word2idx[token])
    enocded_corpus.append(encoded_sentence.copy())
  return enocded_corpus

word2idx = getWord2idx(train + test + valid)
wvecs = np.zeros((len(word2idx), EMBEDDING_DIM), dtype='float32')

with open(f'glove.6B.{EMBEDDING_DIM}d.txt', 'r') as f:
  for line in tqdm(f):
    if len(line.strip().split()) > 3:
      word = line.strip().split()[0]
      if word in word2idx:
        (word, vec) = (word, list(map(float, line.strip().split()[1:])))
        idx = word2idx[word]
        wvecs[idx] = vec

wvecs = wvecs * SCALE_EMBS
wvecs = np.zeros((len(word2idx), EMBEDDING_DIM), dtype='float32')




###LSTMSingleHeadlineModel

####ProcessData

In [ ]:
def generateBatchData(data_id, input_data, data_labels, batch_size):
  batch_data = list()
  single_batch_data = list()
  single_batch_label = list()
  single_batch_id = list()
  max_length = 0
  for num in range(len(input_data)):
    if len(single_batch_data) < batch_size:
      single_batch_data.append(input_data[num].copy())
      single_batch_label.append(data_labels[num])
      single_batch_id.append(data_id[num])
      if len(input_data[num]) > max_length:
        max_length = len(input_data[num])
    else:
      for num in range(len(single_batch_data)):
        while len(single_batch_data[num]) < max_length:
          single_batch_data[num].append(0)
      batch_data.append((single_batch_id.copy(), single_batch_data.copy(), single_batch_label.copy()))
      single_batch_data = list()
      single_batch_label = list()
      single_batch_id = list()
      max_length = 0
      single_batch_data.append(input_data[num].copy())
      single_batch_label.append(data_labels[num])
      single_batch_id.append(data_id[num])
      if len(input_data[num]) > max_length:
        max_length = len(input_data[num])
  if len(single_batch_data) >0:
    for num in range(len(single_batch_data)):
        while len(single_batch_data[num]) < max_length:
          single_batch_data[num].append(0)
    batch_data.append((single_batch_id.copy(), single_batch_data.copy(), single_batch_label.copy()))
  return batch_data

In [ ]:
def get_model_inputs(tokenized_corpus, word2idx, labels, max_len = 0):
  # we index our sentences
  vectorized_sents = [[word2idx[tok] for tok in sent if tok in word2idx] for sent in tokenized_corpus]
  # Sentence lengths
  sent_lengths = [len(sent) for sent in vectorized_sents]
  # Get maximum length
  max_len = max(sent_lengths)
  # if max_len == 0:
  #   max_len = int(max(sent_lengths))
  # we create a tensor of a fixed size filled with zeroes for padding
  sent_tensor = torch.zeros((len(vectorized_sents), max_len)).long().cuda()
  # we fill it with our vectorized sentences 
  for idx, (sent, sentlen) in enumerate(zip(vectorized_sents, sent_lengths)):
    sent_tensor[idx, :sentlen] = torch.LongTensor(sent).cuda()
  # Label tensor
  label_tensor = torch.FloatTensor(labels).cuda()
  
  return sent_tensor, label_tensor, max_len
train_sent_tensor, train_label_tensor, max_len = get_model_inputs(train, word2idx, train_labels)

In [ ]:
encoded_train = encodeData(train, word2idx)
encoded_valid = encodeData(valid, word2idx)
encoded_test = encodeData(test, word2idx)

batch_train = generateBatchData(train_id, encoded_train, train_labels, 24)
batch_test = generateBatchData(test_id, encoded_test, test_labels, 24)
batch_valid = generateBatchData(valid_id, encoded_valid, valid_labels, 24)

train_sent_tensor, train_label_tensor, max_len = get_model_inputs(train, word2idx, train_labels)
test_sent_tensor, test_label_tensor, max_len = get_model_inputs(test, word2idx, test_labels, max_len)

####Train Model

In [ ]:
model = LSTMSingleHeadlineModel(len(wvecs), EMBEDDING_DIM, hidden_dim, out_size, max_length = train_sent_tensor.size()[1] , n_layer = 1, dropout = 0.3, pool_feature_num=200)
model.embedding.weight.data.copy_(torch.from_numpy(wvecs))#

In [ ]:
LRATE = 0.001
EPOCHS = 60
optimizer = optim.Adam(model.parameters(), lr=LRATE)
loss_fn = nn.MSELoss()
feature_train = train_sent_tensor
target_train = train_label_tensor

feature_valid = test_sent_tensor
target_valid = test_label_tensor


feature_test = test_sent_tensor
target_test = test_label_tensor
for epoch in range(1, EPOCHS + 1):
  model.train()
  
  optimizer.zero_grad()

  predictions = model(feature_train).squeeze(1)
  loss = torch.sqrt(loss_fn(predictions, target_train))
  train_loss = loss.item()
  train_ave_error = average_error(predictions, target_train)

  loss.backward()

  optimizer.step()

  model.eval()
  with torch.no_grad():
    predictions_valid = model(feature_valid).squeeze(1)
    valid_loss = torch.sqrt(loss_fn(predictions_valid, target_valid)).item()
    valid_ave_error = average_error(predictions_valid, target_valid)
  print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Train ave_error: {train_ave_error} | Val. Loss: {valid_loss:.3f} | Val. ave_error: {valid_ave_error} |')

####Test

In [ ]:
model.eval()
with torch.no_grad():
  file = open('./LSTMSingleHeadlineModel.csv','w',encoding='utf-8')
  file.writelines('id,pred\n')
  for num in range(len(encoded_test)):
    while len(encoded_test[num]) < 27:
      encoded_test[num].append(0)
    feature_test = torch.LongTensor([encoded_test[num]]).cuda()
    target_test = torch.FloatTensor([test_labels[num]]).cuda()
    predictions = model(feature_test)
    loss = torch.sqrt(loss_fn(predictions, target_test))
    ave_error = average_error(predictions, target_test)
    str_content = test_id[num]  + ',' + str(abs(predictions.item())) + '\n'
    file.writelines(str_content)
  file.close()

###LSTMEditedWordModel

####GetData

In [ ]:
def getCorpus(data_address):
  data_replaced = list()
  data_id = list()
  labels = list()
  data_replace_num = list()
  file = open(data_address,'r',encoding='utf-8')
  file_read = file.readlines()
  flag = 0
  for line in file_read:
    if flag == 0:
      flag = 1
      continue
    line = line.replace('\n','').replace('\r','')
    line = line.split(',')
    if len(line) > 5:
      line_temp = list()
      line_temp.append(line[0])
      line_content = ','.join(line[1:-3])
      line_content_temp = line_content.split(' ')
      for num in range(len(line_content_temp)):
        if '<' in line_content_temp[num] and '/>' in line_content_temp[num]:
          line_content_temp[num] = line[-3]
      line_content = ' '.join(line_content_temp)
      #line_content = pre_process(line_content)
      line_temp.append(line_content)
      line_temp.append(line[-3])
      line_temp.append(line[-2])
      line_temp.append(line[-1])
      line = line_temp
    else:
      line_content_temp = line[1].split(' ')
      for num in range(len(line_content_temp)):
        if '<' in line_content_temp[num] and '/>' in line_content_temp[num]:
          line_content_temp[num] = line[-3]
      line_content = ' '.join(line_content_temp)
      line[1] = line_content
    line_temp = line[1].replace(' ','')
    if '<' in line_temp and '/>' in line_temp:
      p = r"(?<=<).+?(?=/>)" 
      line[1] = line[1].replace('/ >','/>')
      line[1] = re.sub(p, line[2], line[1])
      line[1] = line[1].replace('<','').replace('/>','')
      #line[1] = pre_process(line[1])
    #orginal_data.append(line)
    data_id.append(line[0])
    data_replaced.append(line[1])
    labels.append(float(line[4]))
    data_replace_num_temp = line[2].lower().split(' ')
    data_replace_num_temp2 = list()
    data_replaced_temp = line[1].lower().split(' ')
    
    if len(data_replace_num_temp)>1: 
      for word in data_replace_num_temp:
        if word == '' or word == ' ':
          continue
        data_replace_num_temp2.append(data_replaced_temp.index(word))
    elif len(data_replace_num_temp) == 1:
      data_replace_num_temp2.append(data_replaced_temp.index(data_replace_num_temp[0])) 
    elif len(data_replace_num_temp) == 0:
      print('&&&&&&&&&&&&&&&&&&')
    data_replace_num.append(data_replace_num_temp2.copy())
    if ' ' in line[2]:
      print(line[1],'****',line[2],'$$$$$',data_replace_num_temp2)
  file.close()
  return data_id, data_replaced, data_replace_num, labels

In [ ]:
train_id, train, train_num, train_labels = getCorpus('/content/drive/My Drive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test, test_num, test_labels = getCorpus('/content/drive/My Drive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
valid_id, valid, valid_num, valid_labels = getCorpus('/content/drive/My Drive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')

train = getTokenizedCorpus(train)
test = getTokenizedCorpus(test)
valid = getTokenizedCorpus(valid)

###WordVector

In [ ]:
def getWord2idx(tokenized_corpus):
  vocabulary = []
  for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)
  word2idx = {w: idx+1 for (idx, w) in enumerate(vocabulary)}
  word2idx['<pad>'] = 0
  return word2idx
def encodeData(tokenized_corpus, word2idx):
  enocded_corpus = list()
  for sentence in tokenized_corpus:
    encoded_sentence = list()
    for token in sentence:
      encoded_sentence.append(word2idx[token])
    enocded_corpus.append(encoded_sentence.copy())
  return enocded_corpus

word2idx = getWord2idx(train + test + valid)
wvecs = np.zeros((len(word2idx), EMBEDDING_DIM), dtype='float32')

with open(f'glove.6B.{EMBEDDING_DIM}d.txt', 'r') as f:
  for line in tqdm(f):
    if len(line.strip().split()) > 3:
      word = line.strip().split()[0]
      if word in word2idx:
        (word, vec) = (word, list(map(float, line.strip().split()[1:])))
        idx = word2idx[word]
        wvecs[idx] = vec

wvecs = wvecs * SCALE_EMBS
wvecs = np.zeros((len(word2idx), EMBEDDING_DIM), dtype='float32')




####ProcessData

In [ ]:
def generateBatchData(data_id, input_data, input_data_num, data_labels, batch_size):
  batch_data = list()
  single_batch_data = list()
  single_batch_num = list()
  single_batch_label = list()
  single_batch_id = list()
  max_length = 0
  for num in range(len(input_data)):
    if len(single_batch_data) < batch_size:
      single_batch_data.append(input_data[num].copy())
      single_batch_num.append(input_data_num[num].copy())
      single_batch_label.append(data_labels[num])
      single_batch_id.append(data_id[num])
      if len(input_data[num]) > max_length:
        max_length = len(input_data[num])
    else:
      for num in range(len(single_batch_data)):
        while len(single_batch_data[num]) < max_length:
          single_batch_data[num].append(0)
      batch_data.append((single_batch_id.copy(), single_batch_data.copy(), single_batch_num.copy(), single_batch_label.copy()))
      single_batch_data = list()
      single_batch_label = list()
      single_batch_id = list()
      max_length = 0
      single_batch_data.append(input_data[num].copy())
      single_batch_num.append(input_data_num[num].copy())
      single_batch_label.append(data_labels[num])
      single_batch_id.append(data_id[num])
      if len(input_data[num]) > max_length:
        max_length = len(input_data[num])
  if len(single_batch_data) >0:
    for num in range(len(single_batch_data)):
        while len(single_batch_data[num]) < max_length:
          single_batch_data[num].append(0)
    batch_data.append((single_batch_id.copy(), single_batch_data.copy(), single_batch_num.copy(), single_batch_label.copy()))
  return batch_data

In [ ]:
def get_model_inputs(tokenized_corpus, word2idx, labels, max_len = 0):
  # we index our sentences
  vectorized_sents = [[word2idx[tok] for tok in sent if tok in word2idx] for sent in tokenized_corpus]
  # Sentence lengths
  sent_lengths = [len(sent) for sent in vectorized_sents]
  # Get maximum length
  max_len = max(sent_lengths)
  # if max_len == 0:
  #   max_len = int(max(sent_lengths))
  # we create a tensor of a fixed size filled with zeroes for padding
  sent_tensor = torch.zeros((len(vectorized_sents), max_len)).long().cuda()
  # we fill it with our vectorized sentences 
  for idx, (sent, sentlen) in enumerate(zip(vectorized_sents, sent_lengths)):
    sent_tensor[idx, :sentlen] = torch.LongTensor(sent).cuda()
  # Label tensor
  label_tensor = torch.FloatTensor(labels).cuda()
  
  return sent_tensor, label_tensor, max_len
train_sent_tensor, train_label_tensor, max_len = get_model_inputs(train, word2idx, train_labels)

In [ ]:
encoded_train = encodeData(train, word2idx)
encoded_valid = encodeData(valid, word2idx)
encoded_test = encodeData(test, word2idx)


train_sent_tensor, train_label_tensor, max_len = get_model_inputs(train, word2idx, train_labels)
test_sent_tensor, test_label_tensor, max_len = get_model_inputs(test, word2idx, test_labels, max_len)

In [ ]:
def getEditMatrix(data_sent_tensor,data_num):
  edit_matrix = torch.zeros(data_sent_tensor.size()[0], 1, data_sent_tensor.size()[1])
  edit_total_num = torch.zeros(data_sent_tensor.size()[0], 1, 1)
  
  for batch_ in range(len(data_num)):

    for num in data_num[batch_]:
      edit_matrix[batch_][0][num] = 1
  for batch_ in range(edit_matrix.size()[0]):
    edit_total_num[batch_][0][0] = edit_matrix[batch_].sum()
  return edit_matrix, edit_total_num

####Train Model

In [ ]:
model = LSTMEditedWordModel(len(wvecs), EMBEDDING_DIM, hidden_dim, out_size, max_length = train_sent_tensor.size()[1] , n_layer = 1, dropout = 0.3)
model.embedding.weight.data.copy_(torch.from_numpy(wvecs))#

In [ ]:
LRATE = 0.001
EPOCHS = 60
optimizer = optim.Adam(model.parameters(), lr=LRATE)
loss_fn = nn.MSELoss()
feature_train = train_sent_tensor
target_train = train_label_tensor
train_matrix, train_total_num = getEditMatrix(feature_train,train_num)

feature_valid = test_sent_tensor
target_valid = test_label_tensor
print(len(feature_valid),len(test_num))
valid_matrix, valid_total_num = getEditMatrix(feature_valid,test_num)

feature_test = test_sent_tensor
target_test = test_label_tensor
test_matrix, test_total_num = getEditMatrix(feature_test,test_num)
for epoch in range(1, EPOCHS + 1):
  model.train()
  
  optimizer.zero_grad()

  predictions = model(feature_train, train_matrix, train_total_num).squeeze(1)
  loss = torch.sqrt(loss_fn(predictions, target_train))
  train_loss = loss.item()
  train_ave_error = average_error(predictions, target_train)

  loss.backward()

  optimizer.step()

  model.eval()
  with torch.no_grad():
    predictions_valid = model(feature_valid, valid_matrix, valid_total_num).squeeze(1)
    valid_loss = torch.sqrt(loss_fn(predictions_valid, target_valid)).item()
    valid_ave_error = average_error(predictions_valid, target_valid)
  print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Train ave_error: {train_ave_error} | Val. Loss: {valid_loss:.3f} | Val. ave_error: {valid_ave_error} |')

####Test

In [ ]:
model.eval()
with torch.no_grad():
  file = open('./LSTMEditedWordModel.csv','w',encoding='utf-8')
  file.writelines('id,pred\n')
  for num in range(len(encoded_test)):
    while len(encoded_test[num]) < 27:
      encoded_test[num].append(0)
    feature_test = torch.LongTensor([encoded_test[num]]).cuda()
    target_test = torch.FloatTensor([test_labels[num]]).cuda()
    test_matrix, test_total_num = getEditMatrix(feature_test,[test_num[num]])
    predictions = model(feature_test, test_matrix, test_total_num)
    loss = torch.sqrt(loss_fn(predictions, target_test))
    ave_error = average_error(predictions, target_test)
    str_content = test_id[num]  + ',' + str(abs(predictions.item())) + '\n'
    file.writelines(str_content)
  file.close()

###LSTMTwoHeadlineModel

####GetData

In [ ]:
def pre_process(sentence):
  sentence_temp = sentence.lower()
  sentence_tokenize = word_tokenize(sentence)
  sentence_tokenize = nltk.pos_tag(sentence_tokenize)
  pos_sentence = []
  for token in sentence_tokenize: 
    if token[1] == "NNP" or token[1] == "NN":
      pos_sentence.append(token[0].capitalize())
    else:
      pos_sentence.append(token[0])
  stop_word_removed_sentence = []
  for token in pos_sentence:
    if token not in stop_words_list:
      stop_word_removed_sentence.append(token)
  return_sentence = " ".join(stop_word_removed_sentence)
  return return_sentence

def getCorpus(data_address):
  data_replaced = list()
  data_orginal = list()
  data_orginal_temp = ''
  word_orginal = ''
  data_id = list()
  labels = list()
  file = open(data_address,'r',encoding='utf-8')
  file_read = file.readlines()
  flag = 0
  for line in file_read:
    word_orginal = ''
    if flag == 0:
      flag = 1
      continue
    line = line.replace('\n','').replace('\r','')
    line = line.split(',')
    if len(line) > 5:
      line_temp = list()
      line_temp.append(line[0])
      line_content = ','.join(line[1:-3])
      line_content_temp = line_content.split(' ')
      for num in range(len(line_content_temp)):
        if '<' in line_content_temp[num] and '/>' in line_content_temp[num]:
          word_orginal = line_content_temp[num].replace('<','').replace('/>','')
          line_content_temp[num] = line[-3]
      line_content = ' '.join(line_content_temp)
      line_content = pre_process(line_content)
      line_temp.append(line_content)
      line_temp.append(line[-3])
      line_temp.append(line[-2])
      line_temp.append(line[-1])
      line = line_temp
      
      line[1] = pre_process(line[1])
      data_orginal_temp = line[1].replace(line[2],word_orginal)
    else:
      line_content_temp = line[1].split(' ')
      for num in range(len(line_content_temp)):
        if '<' in line_content_temp[num] and '/>' in line_content_temp[num]:
          word_orginal = line_content_temp[num].replace('<','').replace('/>','')
          line_content_temp[num] = line[-3]
      line_content = ' '.join(line_content_temp)
      line[1] = line_content
      
      line[1] = pre_process(line[1])
      data_orginal_temp = line[1].replace(line[-3],word_orginal)
    line_temp = line[1].replace(' ','')
    if '<' in line_temp and '/>' in line_temp:
      p = r"(?<=<).+?(?=/>)" 
      line[1] = line[1].replace('/ >','/>')
      if word_orginal == '':
        word_orginal = line[1].split('<')[1]
        word_orginal = word_orginal.split('/>')[0]
        #print(line[1],'*************',line[2])
        #print(word_orginal,line[2])
      line[1] = re.sub(p, line[2], line[1])
      line[1] = line[1].replace('<','').replace('/>','')
      
      line[1] = pre_process(line[1])
      data_orginal_temp = line[1].replace(line[2],word_orginal)
      
    print(line[1],'**************',data_orginal_temp)

    #orginal_data.append(line)
    data_id.append(line[0])
    data_orginal.append(data_orginal_temp)
    data_replaced.append(line[1])
    labels.append(float(line[4]))
    
  file.close()
  return data_id, data_orginal, data_replaced, labels
def average_error(output, target):
  output = torch.round(torch.sigmoid(output))
  err_num = 0
  err_mean = 0.0
  err_total = 0.0
  for num in range(len(output)):
    err_total = err_total + abs(output[num].item() - target[num])
  err_mean = float(err_total/len(output))
  return err_mean
def get_tokenized_corpus(corpus):
  tokenized_corpus = []
  for sent in corpus:
    tokenised_sentence = []
    for token in sent.split(' '):
      tokenised_sentence.append(token)
    tokenized_corpus.append(tokenised_sentence)
  return tokenized_corpus
def get_word2idx(tokenized_corpus):
  vocabulary = []
  for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)
  
  word2idx = {w: idx+1 for (idx, w) in enumerate(vocabulary)}
  # we reserve the 0 index for the padding token
  word2idx['<pad>'] = 0

In [ ]:


train_id, train_orginal, train, train_labels = getCorpus('/content/drive/My Drive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test_orginal, test, test_labels = getCorpus('/content/drive/My Drive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
valid_id, valid_orginal, valid, valid_labels = getCorpus('/content/drive/My Drive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')

train = getTokenizedCorpus(train)
train_orginal = getTokenizedCorpus(train_orginal)
test = getTokenizedCorpus(test)
test_orginal = getTokenizedCorpus(test_orginal)
valid = getTokenizedCorpus(valid)
valid_orginal = getTokenizedCorpus(valid_orginal)

####WordVector

In [ ]:
def getWord2idx(tokenized_corpus):
  vocabulary = []
  for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)
  word2idx = {w: idx+1 for (idx, w) in enumerate(vocabulary)}
  word2idx['<pad>'] = 0
  return word2idx
def encodeData(tokenized_corpus, word2idx):
  enocded_corpus = list()
  for sentence in tokenized_corpus:
    encoded_sentence = list()
    for token in sentence:
      encoded_sentence.append(word2idx[token])
    enocded_corpus.append(encoded_sentence.copy())
  return enocded_corpus

word2idx = getWord2idx(train + test + valid + train_orginal + test_orginal + valid_orginal)
wvecs = np.zeros((len(word2idx), EMBEDDING_DIM), dtype='float32')

with open(f'glove.6B.{EMBEDDING_DIM}d.txt', 'r') as f:
  for line in tqdm(f):
    if len(line.strip().split()) > 3:
      word = line.strip().split()[0]
      if word in word2idx:
        (word, vec) = (word, list(map(float, line.strip().split()[1:])))
        idx = word2idx[word]
        wvecs[idx] = vec

wvecs = wvecs * SCALE_EMBS
wvecs = np.zeros((len(word2idx), EMBEDDING_DIM), dtype='float32')




In [ ]:
train

####ProcessData

In [ ]:
def generateBatchData(data_id, input_data, data_labels, batch_size):
  batch_data = list()
  single_batch_data = list()
  single_batch_label = list()
  single_batch_id = list()
  max_length = 0
  for num in range(len(input_data)):
    if len(single_batch_data) < batch_size:
      single_batch_data.append(input_data[num].copy())
      single_batch_label.append(data_labels[num])
      single_batch_id.append(data_id[num])
      if len(input_data[num]) > max_length:
        max_length = len(input_data[num])
    else:
      for num in range(len(single_batch_data)):
        while len(single_batch_data[num]) < max_length:
          single_batch_data[num].append(0)
      batch_data.append((single_batch_id.copy(), single_batch_data.copy(), single_batch_label.copy()))
      single_batch_data = list()
      single_batch_label = list()
      single_batch_id = list()
      max_length = 0
      single_batch_data.append(input_data[num].copy())
      single_batch_label.append(data_labels[num])
      single_batch_id.append(data_id[num])
      if len(input_data[num]) > max_length:
        max_length = len(input_data[num])
  if len(single_batch_data) >0:
    for num in range(len(single_batch_data)):
        while len(single_batch_data[num]) < max_length:
          single_batch_data[num].append(0)
    batch_data.append((single_batch_id.copy(), single_batch_data.copy(), single_batch_label.copy()))
  return batch_data

In [ ]:
def get_model_inputs(tokenized_orginal_corpus, tokenized_corpus, word2idx, labels, max_len = 0):
  # we index our sentences
  vectorized_sents = [[word2idx[tok] for tok in sent if tok in word2idx] for sent in tokenized_corpus]
  vectorized_orginal_sents = [[word2idx[tok] for tok in sent if tok in word2idx] for sent in tokenized_orginal_corpus]
  # Sentence lengths
  sent_lengths = [len(sent) for sent in vectorized_sents]
  sent_orginal_lengths = [len(sent) for sent in vectorized_orginal_sents]
  # Get maximum length
  max_len = max(sent_lengths)
  max_orginal_len = max(sent_orginal_lengths)
  print(max_len)
  # if max_len == 0:
  #   max_len = int(max(sent_lengths))
  # we create a tensor of a fixed size filled with zeroes for padding
  sent_tensor = torch.zeros((len(vectorized_sents), max_len)).long().cuda()
  sent_orginal_tensor = torch.zeros((len(vectorized_orginal_sents), max_orginal_len)).long().cuda()
  # we fill it with our vectorized sentences 
  for idx, (sent, sentlen) in enumerate(zip(vectorized_sents, sent_lengths)):
    
    sent_tensor[idx, :sentlen] = torch.LongTensor(sent).cuda()
  for idx, (sent, sentlen) in enumerate(zip(vectorized_orginal_sents, sent_orginal_lengths)):
    sent_orginal_tensor[idx, :sentlen] = torch.LongTensor(sent).cuda()
  # Label tensor
  label_tensor = torch.FloatTensor(labels).cuda()
  
  return sent_orginal_tensor, sent_tensor, label_tensor, max_len
#train_sent_tensor, train_label_tensor, max_len = get_model_inputs(train_orginal, train, word2idx, train_labels)

In [ ]:
encoded_train = encodeData(train, word2idx)
encoded_valid = encodeData(valid, word2idx)
encoded_test = encodeData(test, word2idx)
encoded_test_orginal = encodeData(test_orginal, word2idx)
# batch_train = generateBatchData(train_id, encoded_train, train_labels, 24)
# batch_test = generateBatchData(test_id, encoded_test, test_labels, 24)
# batch_valid = generateBatchData(valid_id, encoded_valid, valid_labels, 24)

train_sent_orginal_tensor, train_sent_tensor, train_label_tensor, max_len = get_model_inputs(train_orginal, train, word2idx, train_labels)
test_sent_orginal_tensor, test_sent_tensor, test_label_tensor, max_len = get_model_inputs(test_orginal, test, word2idx, test_labels, max_len)

In [ ]:
train

####Train Model

In [ ]:
model = LSTMTwoHeadlineModel(len(wvecs), EMBEDDING_DIM, hidden_dim, out_size, max_length = train_sent_tensor.size()[1] , n_layer = 1, dropout = 0.3)
model.embedding.weight.data.copy_(torch.from_numpy(wvecs))#

In [ ]:
LRATE = 0.001
EPOCHS = 60
optimizer = optim.Adam(model.parameters(), lr=LRATE)
loss_fn = nn.MSELoss()
feature_orginal_train = train_sent_orginal_tensor
feature_train = train_sent_tensor
target_train = train_label_tensor

feature_orginal_valid = test_sent_orginal_tensor
feature_valid = test_sent_tensor
target_valid = test_label_tensor


feature_orginal_test = test_sent_orginal_tensor
feature_test = test_sent_tensor
target_test = test_label_tensor
for epoch in range(1, EPOCHS + 1):
  model.train()
  
  optimizer.zero_grad()

  predictions = model(feature_train, feature_orginal_train).squeeze(1)
  loss = torch.sqrt(loss_fn(predictions, target_train))
  train_loss = loss.item()
  train_ave_error = average_error(predictions, target_train)

  loss.backward()

  optimizer.step()

  model.eval()
  with torch.no_grad():
    predictions_valid = model(feature_valid, feature_orginal_valid).squeeze(1)
    valid_loss = torch.sqrt(loss_fn(predictions_valid, target_valid)).item()
    valid_ave_error = average_error(predictions_valid, target_valid)
  print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Train ave_error: {train_ave_error} | Val. Loss: {valid_loss:.3f} | Val. ave_error: {valid_ave_error} |')

####Test

In [ ]:
model.eval()
with torch.no_grad():
  file = open('./LSTMTwoHeadlineModel.csv','w',encoding='utf-8')
  file.writelines('id,pred\n')
  for num in range(len(encoded_test)):
    while len(encoded_test[num]) < 27:
      encoded_test[num].append(0)
    feature_test = torch.LongTensor([encoded_test[num]]).cuda()
    feature_orginal_test = torch.LongTensor([encoded_test_orginal[num]]).cuda()
    target_test = torch.FloatTensor([test_labels[num]]).cuda()
    predictions = model(feature_test, feature_orginal_test)
    loss = torch.sqrt(loss_fn(predictions, target_test))
    ave_error = average_error(predictions, target_test)
    str_content = test_id[num]  + ',' + str(abs(predictions.item())) + '\n'
    file.writelines(str_content)
  file.close()

###LSTMSequenceClassification

####ProcessData

In [ ]:
def generateBatchData(data_id, input_data, data_labels, batch_size):
  batch_data = list()
  single_batch_data = list()
  single_batch_label = list()
  single_batch_id = list()
  max_length = 0
  for num in range(len(input_data)):
    if len(single_batch_data) < batch_size:
      single_batch_data.append(input_data[num].copy())
      single_batch_label.append(data_labels[num])
      single_batch_id.append(data_id[num])
      if len(input_data[num]) > max_length:
        max_length = len(input_data[num])
    else:
      for num in range(len(single_batch_data)):
        while len(single_batch_data[num]) < max_length:
          single_batch_data[num].append(0)
      batch_data.append((single_batch_id.copy(), single_batch_data.copy(), single_batch_label.copy()))
      single_batch_data = list()
      single_batch_label = list()
      single_batch_id = list()
      max_length = 0
      single_batch_data.append(input_data[num].copy())
      single_batch_label.append(data_labels[num])
      single_batch_id.append(data_id[num])
      if len(input_data[num]) > max_length:
        max_length = len(input_data[num])
  if len(single_batch_data) >0:
    for num in range(len(single_batch_data)):
        while len(single_batch_data[num]) < max_length:
          single_batch_data[num].append(0)
    batch_data.append((single_batch_id.copy(), single_batch_data.copy(), single_batch_label.copy()))
  return batch_data

In [ ]:
def get_model_inputs(tokenized_corpus, word2idx, labels, max_len = 0):
  # we index our sentences
  vectorized_sents = [[word2idx[tok] for tok in sent if tok in word2idx] for sent in tokenized_corpus]
  # Sentence lengths
  sent_lengths = [len(sent) for sent in vectorized_sents]
  # Get maximum length
  max_len = max(sent_lengths)
  # if max_len == 0:
  #   max_len = int(max(sent_lengths))
  # we create a tensor of a fixed size filled with zeroes for padding
  sent_tensor = torch.zeros((len(vectorized_sents), max_len)).long().cuda()
  # we fill it with our vectorized sentences 
  for idx, (sent, sentlen) in enumerate(zip(vectorized_sents, sent_lengths)):
    sent_tensor[idx, :sentlen] = torch.LongTensor(sent).cuda()
  # Label tensor
  label_tensor = torch.FloatTensor(labels).cuda()
  
  return sent_tensor, label_tensor, max_len
train_sent_tensor, train_label_tensor, max_len = get_model_inputs(train, word2idx, train_labels)

In [ ]:
encoded_train = encodeData(train, word2idx)
encoded_valid = encodeData(valid, word2idx)
encoded_test = encodeData(test, word2idx)


batch_train = generateBatchData(train_id, encoded_train, train_labels, 24)
batch_test = generateBatchData(test_id, encoded_test, test_labels, 24)
batch_valid = generateBatchData(valid_id, encoded_valid, valid_labels, 24)

train_sent_tensor, train_label_tensor, max_len = get_model_inputs(train, word2idx, train_labels)
test_sent_tensor, test_label_tensor, max_len = get_model_inputs(test, word2idx, test_labels, max_len)

####Train Model

In [ ]:
model = LSTMAtt(len(wvecs), EMBEDDING_DIM, hidden_dim, out_size, max_length = train_sent_tensor.size()[1] , n_layer = 1, dropout = 0.3)
model.embedding.weight.data.copy_(torch.from_numpy(wvecs))#

In [ ]:
LRATE = 0.001
EPOCHS = 60
optimizer = optim.Adam(model.parameters(), lr=LRATE)
loss_fn = nn.MSELoss()
feature_train = train_sent_tensor
target_train = train_label_tensor

feature_valid = test_sent_tensor
target_valid = test_label_tensor


feature_test = test_sent_tensor
target_test = test_label_tensor
for epoch in range(1, EPOCHS + 1):
  model.train()
  
  optimizer.zero_grad()

  predictions = model(feature_train).squeeze(1)
  loss = torch.sqrt(loss_fn(predictions, target_train))
  train_loss = loss.item()
  train_ave_error = average_error(predictions, target_train)

  loss.backward()

  optimizer.step()

  model.eval()
  with torch.no_grad():
    predictions_valid = model(feature_valid).squeeze(1)
    valid_loss = torch.sqrt(loss_fn(predictions_valid, target_valid)).item()
    valid_ave_error = average_error(predictions_valid, target_valid)
  print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Train ave_error: {train_ave_error} | Val. Loss: {valid_loss:.3f} | Val. ave_error: {valid_ave_error} |')

In [ ]:
feature_train.size()

In [ ]:
batch_test = generateBatchData(test_id, encoded_test, test_labels, 24)

####Test

In [ ]:
model.eval()
with torch.no_grad():
  file = open('./LSTMSequenceClassification.csv','w',encoding='utf-8')
  file.writelines('id,pred\n')
  for num in range(len(encoded_test)):
    while len(encoded_test[num]) < 27:
      encoded_test[num].append(0)
    feature_test = torch.LongTensor([encoded_test[num]]).cuda()
    target_test = torch.FloatTensor([test_labels[num]]).cuda()
    predictions = model(feature_test)
    loss = torch.sqrt(loss_fn(predictions, target_test))
    ave_error = average_error(predictions, target_test)
    str_content = test_id[num]  + ',' + str(abs(predictions.item())) + '\n'
    file.writelines(str_content)
  file.close()

#Test

##Task1

###OurModel

In [ ]:
with torch.no_grad():
  new_title = test_replaced
  old_title = test_orginal
  encoding_old = tokenizer(old_title, return_tensors='pt', padding=True, truncation=True)
  encoding_new = tokenizer(new_title, return_tensors='pt', padding=True, truncation=True)
  input_ids_old = encoding_old['input_ids'].to(device)
  input_ids_new = encoding_new['input_ids'].to(device)
  attention_mask_old = encoding_old['attention_mask'].to(device)
  attention_mask_new = encoding_new['attention_mask'].to(device)
  # outputs = model(input_ids_new, input_ids_old, attention_mask_new, attention_mask_old)[0]
  encoding_index = tokenizer(test_replaced, add_special_tokens=test_editword)['input_ids']
  replace_words = test_editword
  replace_matrix = tokenizer(new_title, return_tensors='pt', padding=True, truncation=True, add_special_tokens=replace_words)['input_ids']
  edit_matrix, edit_total_num = getEditMatrix(tokenizer,encoding_new['input_ids'],encoding_index,replace_matrix,replace_words)
  outputs = model(input_ids_new, attention_mask_new, input_ids_old, attention_mask_old, edit_matrix.to(device), edit_total_num.to(device))
  labels = torch.tensor(test_meanGrade).to(device)
  loss = torch.sqrt(((outputs - labels)**2).mean())
  valid_loss = loss.item()
  print(f'| Val. Loss: {valid_loss:.3f}')


###SingleHeadlineModel

In [ ]:
with torch.no_grad():
  encoding = tokenizer(test_replaced, return_tensors='pt', padding=True, truncation=True)
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  outputs = model(input_ids, attention_mask=attention_mask)
  #outputs = model(input_ids, attention_mask=attention_mask)
  labels = torch.tensor(test_meanGrade).to(device)
  loss = torch.sqrt(loss_fn(outputs,labels))
  valid_loss = loss.item()
  print(f'| Val. Loss: {valid_loss:.3f}')

In [ ]:
input_ids.size()

###EditedWordModel

In [ ]:
with torch.no_grad():
  encoding = tokenizer(test_replaced, return_tensors='pt', padding=True, truncation=True)
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  #outputs = model(input_ids, attention_mask=attention_mask)[0]
  encoding_index = tokenizer(test_replaced, add_special_tokens=test_editword)['input_ids']
  replace_words = test_editword
  replace_matrix = tokenizer(replace_words, return_tensors='pt', padding=True, truncation=True, add_special_tokens=replace_words)['input_ids']
  edit_matrix, edit_total_num = getEditMatrix(tokenizer,encoding['input_ids'],encoding_index,replace_matrix,replace_words)
  outputs = model(input_ids, attention_mask=attention_mask,edit_matrix=edit_matrix.to(device), edit_total_num=edit_total_num.to(device))
  labels = torch.tensor(test_meanGrade).to(device)
  #loss = torch.sqrt(loss_fn(outputs,labels))
  loss = torch.sqrt(((outputs - labels)**2).mean())
  valid_loss = loss.item()
  print(f'| Val. Loss: {valid_loss:.3f}')

###TwoHeadlineModel

In [ ]:
with torch.no_grad():
  new_title = test_replaced
  old_title = test_orginal
  encoding_old = tokenizer(old_title, return_tensors='pt', padding=True, truncation=True)
  encoding_new = tokenizer(new_title, return_tensors='pt', padding=True, truncation=True)
  input_ids_old = encoding_old['input_ids'].to(device)
  input_ids_new = encoding_new['input_ids'].to(device)
  attention_mask_old = encoding_old['attention_mask'].to(device)
  attention_mask_new = encoding_new['attention_mask'].to(device)
  #outputs = model(input_ids_new, input_ids_old, attention_mask_new, attention_mask_old)[0]
  outputs = model(input_ids_new, attention_mask_new, input_ids_old, attention_mask_old)
  labels = torch.tensor(test_meanGrade).to(device)
  loss = torch.sqrt(loss_fn(outputs,labels))
  valid_loss = loss.item()
  print(f'| Val. Loss: {valid_loss:.3f}')

###RoBERTaSequenceClassification

In [ ]:
with torch.no_grad():
  input_title = test_replaced
  input_word = test_editword
  input_data = [[input_title[num_],input_word[num_]] for num_ in range(len(input_title))]
  encoding = tokenizer(input_data, return_tensors='pt', padding=True, truncation=True)
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  outputs = model(input_ids, attention_mask=attention_mask)[0]
  #outputs = model(input_ids, attention_mask=attention_mask)
  labels = torch.tensor(test_meanGrade).to(device)
  loss = torch.sqrt(loss_fn(outputs,labels))
  valid_loss = loss.item()
  print(f'| Val. Loss: {valid_loss:.3f}')

##Task2

###OurModel

In [ ]:
with torch.no_grad():
  #valid_acc_list = list()
  encoding_new1 = tokenizer(test_replaced1, return_tensors='pt', padding=True, truncation=True)
  encoding_old1 = tokenizer(test_orginal1, return_tensors='pt', padding=True, truncation=True)

  encoding_new2 = tokenizer(test_replaced2, return_tensors='pt', padding=True, truncation=True)
  encoding_old2 = tokenizer(test_orginal2, return_tensors='pt', padding=True, truncation=True)

  input_ids_new1 = encoding_new1['input_ids'].to(device)
  input_ids_old1 = encoding_old1['input_ids'].to(device)

  input_ids_new2 = encoding_new2['input_ids'].to(device)
  input_ids_old2 = encoding_old2['input_ids'].to(device)

  attention_mask_new1 = encoding_new1['attention_mask'].to(device)
  attention_mask_old1 = encoding_old1['attention_mask'].to(device)

  attention_mask_new2 = encoding_new2['attention_mask'].to(device)
  attention_mask_old2 = encoding_old2['attention_mask'].to(device)

  new_title1 = test_replaced1
  old_title1 = test_orginal1
  
  new_title2 = test_replaced2
  old_title2 = test_orginal2

  labels = torch.tensor(test_label).to(device)

  encoding_index1 = tokenizer(new_title1, add_special_tokens=test_editword1)['input_ids']
  encoding_index2 = tokenizer(new_title2, add_special_tokens=test_editword2)['input_ids']

  replace_words1 = test_editword1
  replace_words2 = test_editword2

  replace_matrix1 = tokenizer(replace_words1, return_tensors='pt', padding=True, truncation=True, add_special_tokens=data_editword1)['input_ids']
  replace_matrix2 = tokenizer(replace_words2, return_tensors='pt', padding=True, truncation=True, add_special_tokens=data_editword2)['input_ids']

  edit_matrix1, edit_total_num1 = getEditMatrix(tokenizer,encoding_new1['input_ids'],encoding_index1,replace_matrix1,replace_words1)
  edit_matrix2, edit_total_num2 = getEditMatrix(tokenizer,encoding_new2['input_ids'],encoding_index2,replace_matrix2,replace_words2)

  outputs = model(input_ids_new1, attention_mask_new1, input_ids_old1, attention_mask_old1, edit_matrix1.to(device), edit_total_num1.to(device), 
            input_ids_new2, attention_mask_new2, input_ids_old2, attention_mask_old2, edit_matrix2.to(device), edit_total_num2.to(device))
  labels = torch.tensor(test_label[batchStart:batchEnd]).to(device)
  loss = loss_fn(outputs,labels)
  valid_acc = accuracy(outputs, labels)
  valid_loss = loss_fn(outputs,labels)
  batchStart = batchStart + batchSize
  print(f' Val. valid_acc: {valid_acc} |')

###SingleHeadlineModel

In [ ]:
with torch.no_grad():
  encoding = [[test_replaced1[num],test_replaced2[num]]for num in range(len(test_editword1))]
  encoding = tokenizer(encoding, return_tensors='pt', padding=True, truncation=True)
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  outputs = model(input_ids, attention_mask=attention_mask)
  labels = torch.tensor(test_label).to(device)
  valid_acc = accuracy(outputs, labels)
  valid_loss = loss_fn(outputs,labels)
  f_measure(outputs, labels)
  print(f' Val. valid_acc: {valid_acc} |')

###EditedWordModel

In [ ]:
with torch.no_grad():
  old_title = [[data_orginal1[num]+ data_orginal2[num]] for num in range(len(test_meanGrade))]
  new_title = [[data_replaced1[num]+ data_replaced2[num]] for num in range(len(test_meanGrade))]
  title = [[old_title[num], new_title[num]] for num in range(len(old_title))]
  encoding = tokenizer(title return_tensors='pt', padding=True, truncation=True)
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  encoding_index = tokenizer(title, add_special_tokens=data_editword1+data_editword2)['input_ids']
  edit_matrix1, edit_total_num1 = getEditMatrix(tokenizer,encoding['input_ids'],encoding_index,replace_matrix1,replace_words1)
  edit_matrix2, edit_total_num2 = getEditMatrix(tokenizer,encoding['input_ids'],encoding_index,replace_matrix2,replace_words2)
  edit_matrix = edit_matrix1 + edit_matrix2
  edit_total_num = edit_total_num1 + edit_total_num2
  outputs = model(input_ids, attention_mask=attention_mask,edit_matrix=edit_matrix.to(device), edit_total_num=edit_total_num.to(device))
  labels = torch.tensor(test_meanGrade).to(device)
  loss = loss_fn(outputs,labels)
  valid_loss = loss.item()
  print(f' Val. valid_acc: {valid_acc} |')

###TwoHeadlineModel

In [ ]:
with torch.no_grad():
  old_title = [[test_orginal1[num]+'<s>'+test_orginal2[num]] for num in range(len(test_orginal))]
  new_title = [[test_replaced1[num]+'<s>'+test_replaced2[num]] for num in range(len(test_orginal))]
  encoding_old = tokenizer(old_title, return_tensors='pt', padding=True, truncation=True)
  encoding_new = tokenizer(new_title, return_tensors='pt', padding=True, truncation=True)
  input_ids_old = encoding_old['input_ids'].to(device)
  input_ids_new = encoding_new['input_ids'].to(device)
  attention_mask_old = encoding_old['attention_mask'].to(device)
  attention_mask_new = encoding_new['attention_mask'].to(device)
  #outputs = model(input_ids_new, input_ids_old, attention_mask_new, attention_mask_old)[0]
  outputs = model(input_ids_new, attention_mask_new, input_ids_old, attention_mask_old)
  labels = torch.tensor(test_label).to(device)
  valid_acc = accuracy(outputs, labels)
  valid_loss = loss_fn(outputs,labels)
  f_measure(outputs, labels)
  print(f' Val. valid_acc: {valid_acc} |')

###RoBERTaSequenceClassification

In [ ]:
with torch.no_grad():
  encoding = [[test_replaced1[num],test_replaced2[num]]for num in range(len(test_editword1))]
  encoding = tokenizer(encoding, return_tensors='pt', padding=True, truncation=True)
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  outputs = model(input_ids, attention_mask=attention_mask)[0]
  #outputs = model(input_ids, attention_mask=attention_mask)
  labels = torch.tensor(test_label).to(device)
  valid_acc = accuracy(outputs, labels)
  valid_loss = loss_fn(outputs,labels)
  f_measure(outputs, labels)
  print(f' Val. valid_acc: {valid_acc} |')

#ResultSave

##Task1

In [ ]:
#RoBERTaSingleHeadlineModel
#RoBERTaEditedWordModel
#RoBERTaTwoHeadlineModel
#RoBERTaSequenceClassification
#BERTSingleHeadlineModel
#BERTTwoHeadlineModel
#BERTaSequenceClassificatio
import time
time_str = str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
time_str = time_str.replace(' ','-').replace(':','-')
save_name = './' + modelname + time_str + '.csv'
folder = './'
file = open(save_name,'w',encoding='utf-8')
file.writelines('id,pred\n')
for num in range(len(outputs)):
  str_content = test_id[num]  + ',' + str(outputs[num].item()) + '\n'
  file.writelines(str_content)
file.close()

f = zipfile.ZipFile('./'+model_name+'_' + dev_type +'.zip', 'w', zipfile.ZIP_DEFLATED)
f.write(save_name)
f.close()

In [ ]:
outputs.size()

##Task2

In [ ]:

with torch.no_grad():
  file = open('./task-2-output.csv','w',encoding='utf-8')
  file.writelines('id,pred\n')
  for num in range(len(test_id)):
    str_content = str(test_id[num])  + ',' + str(torch.max(torch.softmax(outputs,-1), 1)[1][num].item()) + '\n'
    file.writelines(str_content)
  print(batchEnd,len(test_id))
  file.close()
model_type = 'StackingModel'
f = zipfile.ZipFile('./'+model_name+'_' + dev_type +'.zip', 'w', zipfile.ZIP_DEFLATED)
f.write('./task-2-output.csv')
f.close()

#RoBERTa

In [ ]:
totalEpoch = 2

##Single

In [ ]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from tokenizers import AddedToken
from transformers import RobertaTokenizer
import math
import csv
import re
import os
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn import model_selection
from sklearn import metrics
import transformers
import tokenizers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import RobertaConfig, RobertaModel
#from transformers.modeling_roberta import RobertaClassificationHead
from sklearn.model_selection import train_test_split
from tqdm.autonotebook import tqdm
import utils
from transformers import RobertaConfig, RobertaModel, RobertaForSequenceClassification, RobertaTokenizer
from transformers import BertConfig, BertModel, BertForSequenceClassification, BertTokenizer
from transformers import AdamW
import transformers
import torch
from transformers import get_linear_schedule_with_warmup
import zipfile
import random
def fix_seed(seed=1):
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)
fix_seed()
configuration = RobertaConfig()
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=True)
dev_type = 'No_dev'
import csv
import re
def readTask1Data(data_address):
  data_id = list()
  data_mask = list()
  data_orginal = list()
  data_replaced = list()
  data_grades = list()
  data_meanGrade = list()
  data_editword = list()
  data_orginalword = list()
  flag = 0
  p = r"(?<=<).+?(?=/>)" 
  with open(data_address, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      data_id.append(row[0])
      line = row[1]
      while '  ' in line:
        line.replace('  ',' ')
      line = line.replace('/ >','/>')
      line = line.replace('< ','<')
      line = line.replace(' />','/>')
      data_orginalword.append(re.findall(p, line))
      data_orginal_temp = line.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal.append(data_orginal_temp)
      data_mask_temp = re.sub(p, 'edit', line)
      if '<edit/>' not in data_mask_temp:
        print(data_mask_temp)
      data_mask.append(data_mask_temp)
      data_replaced_temp = data_mask_temp.replace('<edit/>',row[2])
      data_replaced.append(data_replaced_temp)
      data_grades.append(row[3])
      data_meanGrade.append([float(row[4])])
      data_editword.append(row[2])
  return  data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test_mask, test_orginal, test_replaced, test_grades, test_meanGrade, test_editword, test_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
def splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, valid_num=0.3):
  train_id = list()
  train_mask = list()
  train_orginal = list()
  train_replaced = list()
  train_grades = list()
  train_meanGrade = list()
  train_editword = list()
  train_orginalword = list()
  val_id = list()
  val_mask = list()
  val_orginal = list()
  val_replaced = list()
  val_grades = list()
  val_meanGrade = list()
  val_editword = list()
  val_orginalword = list()
  temp_list = list()
  for num in range(len(data_id)):
    temp_list.append([data_id[num], data_mask[num], data_orginal[num], data_replaced[num], data_grades[num], data_meanGrade[num], data_editword[num], data_orginalword[num]])
  train_set,valid_set = train_test_split(temp_list,test_size = valid_num)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in train_set:
    train_id.append(set_id_temp)
    train_mask.append(set_mask_temp)
    train_orginal.append(set_orginal_temp)
    train_replaced.append(set_replaced_temp)
    train_grades.append(set_grades_temp)
    train_meanGrade.append(set_meanGrade_temp)
    train_editword.append(set_editword_temp)
    train_orginalword.append(set_orginalword_temp)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in valid_set:
    val_id.append(set_id_temp)
    val_mask.append(set_mask_temp)
    val_orginal.append(set_orginal_temp)
    val_replaced.append(set_replaced_temp)
    val_grades.append(set_grades_temp)
    val_meanGrade.append(set_meanGrade_temp)
    val_editword.append(set_editword_temp)
    val_orginalword.append(set_orginalword_temp)
  return train_id, train_mask, train_orginal, train_replaced, train_grades, train_meanGrade, train_editword, train_orginalword, val_id,val_mask,val_orginal,val_replaced,val_grades,val_meanGrade,val_editword,val_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, val_id, val_mask, val_orginal, val_replaced, val_grades, val_meanGrade, val_editword, val_orginalword = splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword)
dev_type = 'No_dev'
import csv
import re
def readTask1Data(data_address):
  data_id = list()
  data_mask = list()
  data_orginal = list()
  data_replaced = list()
  data_grades = list()
  data_meanGrade = list()
  data_editword = list()
  data_orginalword = list()
  flag = 0
  p = r"(?<=<).+?(?=/>)" 
  with open(data_address, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      data_id.append(row[0])
      line = row[1]
      while '  ' in line:
        line.replace('  ',' ')
      line = line.replace('/ >','/>')
      line = line.replace('< ','<')
      line = line.replace(' />','/>')
      data_orginalword.append(re.findall(p, line))
      data_orginal_temp = line.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal.append(data_orginal_temp)
      data_mask_temp = re.sub(p, 'edit', line)
      if '<edit/>' not in data_mask_temp:
        print(data_mask_temp)
      data_mask.append(data_mask_temp)
      data_replaced_temp = data_mask_temp.replace('<edit/>',row[2])
      data_replaced.append(data_replaced_temp)
      data_grades.append(row[3])
      data_meanGrade.append([float(row[4])])
      data_editword.append(row[2])
  return  data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test_mask, test_orginal, test_replaced, test_grades, test_meanGrade, test_editword, test_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
def splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, valid_num=0.3):
  train_id = list()
  train_mask = list()
  train_orginal = list()
  train_replaced = list()
  train_grades = list()
  train_meanGrade = list()
  train_editword = list()
  train_orginalword = list()
  val_id = list()
  val_mask = list()
  val_orginal = list()
  val_replaced = list()
  val_grades = list()
  val_meanGrade = list()
  val_editword = list()
  val_orginalword = list()
  temp_list = list()
  for num in range(len(data_id)):
    temp_list.append([data_id[num], data_mask[num], data_orginal[num], data_replaced[num], data_grades[num], data_meanGrade[num], data_editword[num], data_orginalword[num]])
  train_set,valid_set = train_test_split(temp_list,test_size = valid_num)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in train_set:
    train_id.append(set_id_temp)
    train_mask.append(set_mask_temp)
    train_orginal.append(set_orginal_temp)
    train_replaced.append(set_replaced_temp)
    train_grades.append(set_grades_temp)
    train_meanGrade.append(set_meanGrade_temp)
    train_editword.append(set_editword_temp)
    train_orginalword.append(set_orginalword_temp)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in valid_set:
    val_id.append(set_id_temp)
    val_mask.append(set_mask_temp)
    val_orginal.append(set_orginal_temp)
    val_replaced.append(set_replaced_temp)
    val_grades.append(set_grades_temp)
    val_meanGrade.append(set_meanGrade_temp)
    val_editword.append(set_editword_temp)
    val_orginalword.append(set_orginalword_temp)
  return train_id, train_mask, train_orginal, train_replaced, train_grades, train_meanGrade, train_editword, train_orginalword, val_id,val_mask,val_orginal,val_replaced,val_grades,val_meanGrade,val_editword,val_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, val_id, val_mask, val_orginal, val_replaced, val_grades, val_meanGrade, val_editword, val_orginalword = splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword)
class editHeadlineModel(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(editHeadlineModel, self).__init__(conf)
    self.roberta = transformers.RobertaModel.from_pretrained('roberta-base')
    self.drop_out = nn.Dropout(0.1)
    #self.pooling = nn.AdaptiveAvgPool1d(1024)
    self.dense = nn.Linear(768, 768)
    self.linear = nn.Linear(768, 1)
  def forward(self, input_ids, attention_mask):
    out = self.roberta(input_ids,attention_mask=attention_mask)[0]
    out = out[:,0,:]
    out = self.drop_out(out)
    out = torch.tanh(out)
    out = self.drop_out(out)
    out = self.linear(out)
    return out
model = editHeadlineModel(configuration)
model_name = 'editHeadlineModel'
lr = 3e-5
max_grad_norm = 1.0
eps = 1e-8
batchSize = 16
num_train_steps = totalEpoch
WU = 0.2
num_training_steps = ((len(data_id)/batchSize)+1) * totalEpoch
num_warmup_steps = int(num_training_steps * WU)
#scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)
optimizer = AdamW(model.parameters(), lr = lr, eps = eps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_train_steps, num_training_steps=num_training_steps)
loss_fn = nn.MSELoss()
#loss_fn = nn.CrossEntropyLoss()##Task2
device = torch.device("cuda")
model.to(device)
for epoch in range(totalEpoch):
  batchStart = 0
  train_loss = 0
  optimizer.zero_grad()
  while batchStart <= len(data_meanGrade):
    batchEnd = batchStart + batchSize
    if batchEnd > len(data_meanGrade):
      batchEnd = len(data_meanGrade)
    encoding = tokenizer(data_replaced[batchStart:batchEnd], return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    labels = torch.tensor(data_meanGrade[batchStart:batchEnd]).to(device)
    model.train()
    outputs = model(input_ids, attention_mask=attention_mask)
    #outputs = model(input_ids, attention_mask=attention_mask)
    #loss = torch.sqrt(loss_fn(outputs,labels))
    loss = torch.sqrt(((outputs - labels)**2).mean())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    batchStart = batchStart + batchSize
  with torch.no_grad():
    encoding = tokenizer(val_replaced, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)
    #outputs = model(input_ids, attention_mask=attention_mask)
    labels = torch.tensor(val_meanGrade).to(device)
    loss = torch.sqrt(loss_fn(outputs,labels))
    valid_loss = loss.item()
    print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')
with torch.no_grad():
  encoding = tokenizer(test_replaced, return_tensors='pt', padding=True, truncation=True)
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  outputs = model(input_ids, attention_mask=attention_mask)
  #outputs = model(input_ids, attention_mask=attention_mask)
  labels = torch.tensor(test_meanGrade).to(device)
  loss = torch.sqrt(loss_fn(outputs,labels))
  valid_loss = loss.item()
  print(f'| Val. Loss: {valid_loss:.3f}')
#RoBERTaSingleHeadlineModel
#RoBERTaEditedWordModel
#RoBERTaTwoHeadlineModel
#RoBERTaSequenceClassification
#BERTSingleHeadlineModel
#BERTTwoHeadlineModel
#BERTaSequenceClassificatio
import time
model2 = 'RoBERTaSingleHeadlineModel'
time_str = str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
time_str = time_str.replace(' ','-').replace(':','-')
save_name = './' + model2 + str(totalEpoch) + '-' + time_str + '.csv'
folder = './'
file = open(save_name, 'w', encoding='utf-8')
file.writelines('id,pred\n')
for num in range(len(outputs)):
  str_content = test_id[num]  + ',' + str(outputs[num].item()) + '\n'
  file.writelines(str_content)
file.close()

f = zipfile.ZipFile('./'+model_name+'_' + dev_type +'.zip', 'w', zipfile.ZIP_DEFLATED)
f.write(save_name)
f.close()

In [ ]:
totalEpoch = 2

##Edited

In [ ]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from tokenizers import AddedToken
from transformers import RobertaTokenizer
import math
import csv
import re
import os
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn import model_selection
from sklearn import metrics
import transformers
import tokenizers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import RobertaConfig, RobertaModel
#from transformers.modeling_roberta import RobertaClassificationHead
from sklearn.model_selection import train_test_split
from tqdm.autonotebook import tqdm
import utils
from transformers import RobertaConfig, RobertaModel, RobertaForSequenceClassification, RobertaTokenizer
from transformers import BertConfig, BertModel, BertForSequenceClassification, BertTokenizer
from transformers import AdamW
import transformers
import torch
from transformers import get_linear_schedule_with_warmup
import zipfile
import random
def fix_seed(seed=1):
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)
fix_seed()
configuration = RobertaConfig()
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=True)
dev_type = 'No_dev'
import csv
import re
def readTask1Data(data_address):
  data_id = list()
  data_mask = list()
  data_orginal = list()
  data_replaced = list()
  data_grades = list()
  data_meanGrade = list()
  data_editword = list()
  data_orginalword = list()
  flag = 0
  p = r"(?<=<).+?(?=/>)" 
  with open(data_address, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      data_id.append(row[0])
      line = row[1]
      while '  ' in line:
        line.replace('  ',' ')
      line = line.replace('/ >','/>')
      line = line.replace('< ','<')
      line = line.replace(' />','/>')
      data_orginalword.append(re.findall(p, line))
      data_orginal_temp = line.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal.append(data_orginal_temp)
      data_mask_temp = re.sub(p, 'edit', line)
      if '<edit/>' not in data_mask_temp:
        print(data_mask_temp)
      data_mask.append(data_mask_temp)
      data_replaced_temp = data_mask_temp.replace('<edit/>',row[2])
      data_replaced.append(data_replaced_temp)
      data_grades.append(row[3])
      data_meanGrade.append([float(row[4])])
      data_editword.append(row[2])
  return  data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test_mask, test_orginal, test_replaced, test_grades, test_meanGrade, test_editword, test_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
def splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, valid_num=0.3):
  train_id = list()
  train_mask = list()
  train_orginal = list()
  train_replaced = list()
  train_grades = list()
  train_meanGrade = list()
  train_editword = list()
  train_orginalword = list()
  val_id = list()
  val_mask = list()
  val_orginal = list()
  val_replaced = list()
  val_grades = list()
  val_meanGrade = list()
  val_editword = list()
  val_orginalword = list()
  temp_list = list()
  for num in range(len(data_id)):
    temp_list.append([data_id[num], data_mask[num], data_orginal[num], data_replaced[num], data_grades[num], data_meanGrade[num], data_editword[num], data_orginalword[num]])
  train_set,valid_set = train_test_split(temp_list,test_size = valid_num)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in train_set:
    train_id.append(set_id_temp)
    train_mask.append(set_mask_temp)
    train_orginal.append(set_orginal_temp)
    train_replaced.append(set_replaced_temp)
    train_grades.append(set_grades_temp)
    train_meanGrade.append(set_meanGrade_temp)
    train_editword.append(set_editword_temp)
    train_orginalword.append(set_orginalword_temp)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in valid_set:
    val_id.append(set_id_temp)
    val_mask.append(set_mask_temp)
    val_orginal.append(set_orginal_temp)
    val_replaced.append(set_replaced_temp)
    val_grades.append(set_grades_temp)
    val_meanGrade.append(set_meanGrade_temp)
    val_editword.append(set_editword_temp)
    val_orginalword.append(set_orginalword_temp)
  return train_id, train_mask, train_orginal, train_replaced, train_grades, train_meanGrade, train_editword, train_orginalword, val_id,val_mask,val_orginal,val_replaced,val_grades,val_meanGrade,val_editword,val_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, val_id, val_mask, val_orginal, val_replaced, val_grades, val_meanGrade, val_editword, val_orginalword = splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword)
dev_type = 'No_dev'
import csv
import re
def readTask1Data(data_address):
  data_id = list()
  data_mask = list()
  data_orginal = list()
  data_replaced = list()
  data_grades = list()
  data_meanGrade = list()
  data_editword = list()
  data_orginalword = list()
  flag = 0
  p = r"(?<=<).+?(?=/>)" 
  with open(data_address, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      data_id.append(row[0])
      line = row[1]
      while '  ' in line:
        line.replace('  ',' ')
      line = line.replace('/ >','/>')
      line = line.replace('< ','<')
      line = line.replace(' />','/>')
      data_orginalword.append(re.findall(p, line))
      data_orginal_temp = line.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal.append(data_orginal_temp)
      data_mask_temp = re.sub(p, 'edit', line)
      if '<edit/>' not in data_mask_temp:
        print(data_mask_temp)
      data_mask.append(data_mask_temp)
      data_replaced_temp = data_mask_temp.replace('<edit/>',row[2])
      data_replaced.append(data_replaced_temp)
      data_grades.append(row[3])
      data_meanGrade.append([float(row[4])])
      data_editword.append(row[2])
  return  data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test_mask, test_orginal, test_replaced, test_grades, test_meanGrade, test_editword, test_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
def splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, valid_num=0.3):
  train_id = list()
  train_mask = list()
  train_orginal = list()
  train_replaced = list()
  train_grades = list()
  train_meanGrade = list()
  train_editword = list()
  train_orginalword = list()
  val_id = list()
  val_mask = list()
  val_orginal = list()
  val_replaced = list()
  val_grades = list()
  val_meanGrade = list()
  val_editword = list()
  val_orginalword = list()
  temp_list = list()
  for num in range(len(data_id)):
    temp_list.append([data_id[num], data_mask[num], data_orginal[num], data_replaced[num], data_grades[num], data_meanGrade[num], data_editword[num], data_orginalword[num]])
  train_set,valid_set = train_test_split(temp_list,test_size = valid_num)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in train_set:
    train_id.append(set_id_temp)
    train_mask.append(set_mask_temp)
    train_orginal.append(set_orginal_temp)
    train_replaced.append(set_replaced_temp)
    train_grades.append(set_grades_temp)
    train_meanGrade.append(set_meanGrade_temp)
    train_editword.append(set_editword_temp)
    train_orginalword.append(set_orginalword_temp)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in valid_set:
    val_id.append(set_id_temp)
    val_mask.append(set_mask_temp)
    val_orginal.append(set_orginal_temp)
    val_replaced.append(set_replaced_temp)
    val_grades.append(set_grades_temp)
    val_meanGrade.append(set_meanGrade_temp)
    val_editword.append(set_editword_temp)
    val_orginalword.append(set_orginalword_temp)
  return train_id, train_mask, train_orginal, train_replaced, train_grades, train_meanGrade, train_editword, train_orginalword, val_id,val_mask,val_orginal,val_replaced,val_grades,val_meanGrade,val_editword,val_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, val_id, val_mask, val_orginal, val_replaced, val_grades, val_meanGrade, val_editword, val_orginalword = splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword)
class wordModel(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(wordModel, self).__init__(conf)
    self.roberta = transformers.RobertaModel.from_pretrained('roberta-base')
    self.drop_out = nn.Dropout(0.1)
    self.dense = nn.Linear(768, 768)
    self.linear = nn.Linear(768, 1)
  def forward(self, input_ids, attention_mask, edit_matrix, edit_total_num):
    batch_size = input_ids.size()[0]
    out = self.roberta(input_ids,attention_mask=attention_mask)[0]
    seq_size = out.size()[1]
    #print(edit_matrix.size(), out.size())
    out = torch.bmm(edit_matrix, out)
    #print(out.size(),out.squeeze(dim=1).size())
    out = out/edit_total_num
    out = out.squeeze(dim=1)
    out = self.drop_out(out)
    out = torch.tanh(out)
    out = self.drop_out(out)
    logits = self.linear(out)
    return logits
model = wordModel(configuration)
model_name = 'wordModel'
lr = 3e-5
max_grad_norm = 1.0
eps = 1e-8
batchSize = 16
num_train_steps = totalEpoch
WU = 0.2
num_training_steps = ((len(data_id)/batchSize)+1) * totalEpoch
num_warmup_steps = int(num_training_steps * WU)
#scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)
optimizer = AdamW(model.parameters(), lr = lr, eps = eps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_train_steps, num_training_steps=num_training_steps)
loss_fn = nn.MSELoss()
#loss_fn = nn.CrossEntropyLoss()##Task2
def getEditMatrix(tokenizer, encoding_sentence, encoding_index, replace_matrix, replace_words):
  edit_matrix = torch.zeros(encoding_sentence.size()[0], 1, encoding_sentence.size()[1])
  edit_total_num = torch.zeros(encoding_sentence.size()[0], 1, 1)
  for batch_ in range(len(replace_words)):
    token_list = tokenizer.convert_ids_to_tokens(encoding_index[batch_][1:-1])
    token_str= " ".join(token_list)
    token_str = token_str.split('Ġ')
    token_dict = dict()
    for temp in token_str:
      token_dict[temp.replace(' ','')] = ('Ġ' + temp).split(' ')
    for key in token_dict:
      if replace_words[batch_].lower() in key.lower() and float(len(replace_words[batch_])/len(key)) > 0.5:
        for tok_num in range(len(token_dict[key])):
          if len(token_dict[key]) == 2 and token_dict[key][0].replace('Ġ','') in token_list:
            edit_matrix[batch_][0][encoding_index[batch_].index(tokenizer.convert_tokens_to_ids(token_dict[key][0].replace('Ġ','')))] = 1
            continue
          if token_dict[key][tok_num] == '':
            continue
          edit_token_ids = tokenizer.convert_tokens_to_ids(token_dict[key][tok_num])
          edit_matrix[batch_][0][encoding_index[batch_].index(tokenizer.convert_tokens_to_ids(token_dict[key][tok_num]))] = 1
  for batch_ in range(edit_matrix.size()[0]):
    edit_total_num[batch_][0][0] = edit_matrix[batch_].sum()
  return edit_matrix, edit_total_num
device = torch.device("cuda")
model.to(device)
for epoch in range(totalEpoch):
  batchStart = 0
  train_loss = 0
  optimizer.zero_grad()
  while batchStart <= len(data_meanGrade):
    batchEnd = batchStart + batchSize
    if batchEnd > len(data_meanGrade):
      batchEnd = len(data_meanGrade)
    encoding = tokenizer(data_replaced[batchStart:batchEnd], return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    labels = torch.tensor(data_meanGrade[batchStart:batchEnd]).to(device)
    encoding_index = tokenizer(data_replaced[batchStart:batchEnd], add_special_tokens=data_editword)['input_ids']
    replace_words = data_editword[batchStart:batchEnd]
    replace_matrix = tokenizer(replace_words, return_tensors='pt', padding=True, truncation=True, add_special_tokens=data_editword)['input_ids']
    edit_matrix, edit_total_num = getEditMatrix(tokenizer,encoding['input_ids'],encoding_index,replace_matrix,replace_words)
    model.train()
    #outputs = model(input_ids, attention_mask=attention_mask)[0]
    outputs = model(input_ids, attention_mask=attention_mask,edit_matrix=edit_matrix.to(device), edit_total_num=edit_total_num.to(device))
    #loss = torch.sqrt(loss_fn(outputs,labels))
    loss = torch.sqrt(((outputs - labels)**2).mean())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    batchStart = batchStart + batchSize
    
  with torch.no_grad():
    encoding = tokenizer(val_replaced, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    #outputs = model(input_ids, attention_mask=attention_mask)[0]
    encoding_index = tokenizer(val_replaced, add_special_tokens=val_editword)['input_ids']
    replace_words = val_editword
    replace_matrix = tokenizer(replace_words, return_tensors='pt', padding=True, truncation=True, add_special_tokens=replace_words)['input_ids']
    edit_matrix, edit_total_num = getEditMatrix(tokenizer,encoding['input_ids'],encoding_index,replace_matrix,replace_words)
    outputs = model(input_ids, attention_mask=attention_mask,edit_matrix=edit_matrix.to(device), edit_total_num=edit_total_num.to(device))
    labels = torch.tensor(val_meanGrade).to(device)
    #loss = torch.sqrt(loss_fn(outputs,labels))
    loss = torch.sqrt(((outputs - labels)**2).mean())
    valid_loss = loss.item()
    print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')
with torch.no_grad():
  encoding = tokenizer(test_replaced, return_tensors='pt', padding=True, truncation=True)
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  #outputs = model(input_ids, attention_mask=attention_mask)[0]
  encoding_index = tokenizer(test_replaced, add_special_tokens=test_editword)['input_ids']
  replace_words = test_editword
  replace_matrix = tokenizer(replace_words, return_tensors='pt', padding=True, truncation=True, add_special_tokens=replace_words)['input_ids']
  edit_matrix, edit_total_num = getEditMatrix(tokenizer,encoding['input_ids'],encoding_index,replace_matrix,replace_words)
  outputs = model(input_ids, attention_mask=attention_mask,edit_matrix=edit_matrix.to(device), edit_total_num=edit_total_num.to(device))
  labels = torch.tensor(test_meanGrade).to(device)
  #loss = torch.sqrt(loss_fn(outputs,labels))
  loss = torch.sqrt(((outputs - labels)**2).mean())
  valid_loss = loss.item()
  print(f'| Val. Loss: {valid_loss:.3f}')
#RoBERTaSingleHeadlineModel
#RoBERTaEditedWordModel
#RoBERTaTwoHeadlineModel
#RoBERTaSequenceClassification
#BERTSingleHeadlineModel
#BERTTwoHeadlineModel
#BERTaSequenceClassificatio
import time
model2 = 'RoBERTaEditedWordModel'
time_str = str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
time_str = time_str.replace(' ','-').replace(':','-')
save_name = './' + model2 + str(totalEpoch) + '-' + time_str + '.csv'
folder = './'
file = open(save_name, 'w', encoding='utf-8')
file.writelines('id,pred\n')
for num in range(len(outputs)):
  str_content = test_id[num]  + ',' + str(outputs[num].item()) + '\n'
  file.writelines(str_content)
file.close()

f = zipfile.ZipFile('./'+model_name+'_' + dev_type +'.zip', 'w', zipfile.ZIP_DEFLATED)
f.write(save_name)
f.close()

##TWO

In [ ]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from tokenizers import AddedToken
from transformers import RobertaTokenizer
import math
import csv
import re
import os
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn import model_selection
from sklearn import metrics
import transformers
import tokenizers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import RobertaConfig, RobertaModel
#from transformers.modeling_roberta import RobertaClassificationHead
from sklearn.model_selection import train_test_split
from tqdm.autonotebook import tqdm
import utils
from transformers import RobertaConfig, RobertaModel, RobertaForSequenceClassification, RobertaTokenizer
from transformers import BertConfig, BertModel, BertForSequenceClassification, BertTokenizer
from transformers import AdamW
import transformers
import torch
from transformers import get_linear_schedule_with_warmup
import zipfile
import random
def fix_seed(seed=1):
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)
fix_seed()
configuration = RobertaConfig()
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=True)
dev_type = 'No_dev'
import csv
import re
def readTask1Data(data_address):
  data_id = list()
  data_mask = list()
  data_orginal = list()
  data_replaced = list()
  data_grades = list()
  data_meanGrade = list()
  data_editword = list()
  data_orginalword = list()
  flag = 0
  p = r"(?<=<).+?(?=/>)" 
  with open(data_address, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      data_id.append(row[0])
      line = row[1]
      while '  ' in line:
        line.replace('  ',' ')
      line = line.replace('/ >','/>')
      line = line.replace('< ','<')
      line = line.replace(' />','/>')
      data_orginalword.append(re.findall(p, line))
      data_orginal_temp = line.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal.append(data_orginal_temp)
      data_mask_temp = re.sub(p, 'edit', line)
      if '<edit/>' not in data_mask_temp:
        print(data_mask_temp)
      data_mask.append(data_mask_temp)
      data_replaced_temp = data_mask_temp.replace('<edit/>',row[2])
      data_replaced.append(data_replaced_temp)
      data_grades.append(row[3])
      data_meanGrade.append([float(row[4])])
      data_editword.append(row[2])
  return  data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test_mask, test_orginal, test_replaced, test_grades, test_meanGrade, test_editword, test_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
def splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, valid_num=0.3):
  train_id = list()
  train_mask = list()
  train_orginal = list()
  train_replaced = list()
  train_grades = list()
  train_meanGrade = list()
  train_editword = list()
  train_orginalword = list()
  val_id = list()
  val_mask = list()
  val_orginal = list()
  val_replaced = list()
  val_grades = list()
  val_meanGrade = list()
  val_editword = list()
  val_orginalword = list()
  temp_list = list()
  for num in range(len(data_id)):
    temp_list.append([data_id[num], data_mask[num], data_orginal[num], data_replaced[num], data_grades[num], data_meanGrade[num], data_editword[num], data_orginalword[num]])
  train_set,valid_set = train_test_split(temp_list,test_size = valid_num)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in train_set:
    train_id.append(set_id_temp)
    train_mask.append(set_mask_temp)
    train_orginal.append(set_orginal_temp)
    train_replaced.append(set_replaced_temp)
    train_grades.append(set_grades_temp)
    train_meanGrade.append(set_meanGrade_temp)
    train_editword.append(set_editword_temp)
    train_orginalword.append(set_orginalword_temp)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in valid_set:
    val_id.append(set_id_temp)
    val_mask.append(set_mask_temp)
    val_orginal.append(set_orginal_temp)
    val_replaced.append(set_replaced_temp)
    val_grades.append(set_grades_temp)
    val_meanGrade.append(set_meanGrade_temp)
    val_editword.append(set_editword_temp)
    val_orginalword.append(set_orginalword_temp)
  return train_id, train_mask, train_orginal, train_replaced, train_grades, train_meanGrade, train_editword, train_orginalword, val_id,val_mask,val_orginal,val_replaced,val_grades,val_meanGrade,val_editword,val_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, val_id, val_mask, val_orginal, val_replaced, val_grades, val_meanGrade, val_editword, val_orginalword = splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword)
dev_type = 'No_dev'
import csv
import re
def readTask1Data(data_address):
  data_id = list()
  data_mask = list()
  data_orginal = list()
  data_replaced = list()
  data_grades = list()
  data_meanGrade = list()
  data_editword = list()
  data_orginalword = list()
  flag = 0
  p = r"(?<=<).+?(?=/>)" 
  with open(data_address, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      data_id.append(row[0])
      line = row[1]
      while '  ' in line:
        line.replace('  ',' ')
      line = line.replace('/ >','/>')
      line = line.replace('< ','<')
      line = line.replace(' />','/>')
      data_orginalword.append(re.findall(p, line))
      data_orginal_temp = line.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal.append(data_orginal_temp)
      data_mask_temp = re.sub(p, 'edit', line)
      if '<edit/>' not in data_mask_temp:
        print(data_mask_temp)
      data_mask.append(data_mask_temp)
      data_replaced_temp = data_mask_temp.replace('<edit/>',row[2])
      data_replaced.append(data_replaced_temp)
      data_grades.append(row[3])
      data_meanGrade.append([float(row[4])])
      data_editword.append(row[2])
  return  data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test_mask, test_orginal, test_replaced, test_grades, test_meanGrade, test_editword, test_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
def splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, valid_num=0.3):
  train_id = list()
  train_mask = list()
  train_orginal = list()
  train_replaced = list()
  train_grades = list()
  train_meanGrade = list()
  train_editword = list()
  train_orginalword = list()
  val_id = list()
  val_mask = list()
  val_orginal = list()
  val_replaced = list()
  val_grades = list()
  val_meanGrade = list()
  val_editword = list()
  val_orginalword = list()
  temp_list = list()
  for num in range(len(data_id)):
    temp_list.append([data_id[num], data_mask[num], data_orginal[num], data_replaced[num], data_grades[num], data_meanGrade[num], data_editword[num], data_orginalword[num]])
  train_set,valid_set = train_test_split(temp_list,test_size = valid_num)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in train_set:
    train_id.append(set_id_temp)
    train_mask.append(set_mask_temp)
    train_orginal.append(set_orginal_temp)
    train_replaced.append(set_replaced_temp)
    train_grades.append(set_grades_temp)
    train_meanGrade.append(set_meanGrade_temp)
    train_editword.append(set_editword_temp)
    train_orginalword.append(set_orginalword_temp)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in valid_set:
    val_id.append(set_id_temp)
    val_mask.append(set_mask_temp)
    val_orginal.append(set_orginal_temp)
    val_replaced.append(set_replaced_temp)
    val_grades.append(set_grades_temp)
    val_meanGrade.append(set_meanGrade_temp)
    val_editword.append(set_editword_temp)
    val_orginalword.append(set_orginalword_temp)
  return train_id, train_mask, train_orginal, train_replaced, train_grades, train_meanGrade, train_editword, train_orginalword, val_id,val_mask,val_orginal,val_replaced,val_grades,val_meanGrade,val_editword,val_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, val_id, val_mask, val_orginal, val_replaced, val_grades, val_meanGrade, val_editword, val_orginalword = splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword)
class twoHeadlineModel_v1(transformers.BertPreTrainedModel):
  def __init__(self, conf):
    super(twoHeadlineModel_v1, self).__init__(conf)
    self.roberta = transformers.RobertaModel.from_pretrained('roberta-base')
    #self.roberta = transformers.RobertaModel('roberta-base')
    self.dense = nn.Linear(768*4, 768*4)
    self.dropout = nn.Dropout(0.1)
    self.linear = nn.Linear(768*4,1)
  def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
    out1 = self.roberta(input_ids1,attention_mask=attention_mask1)[0]
    out2 = self.roberta(input_ids2,attention_mask=attention_mask2)[0]
    out1 = out1[:,0,:]
    out2 = out2[:,0,:]
    merge = [out1, out2, (out1 - out2).abs(), out1 * out2]
    cat = torch.cat(merge, dim=-1)
    out = self.dense(cat)
    logits = self.linear(out)
    return logits
model = twoHeadlineModel_v1(configuration)
model_name = 'twoHeadlineModel_v1'
lr = 3e-5
max_grad_norm = 1.0
eps = 1e-8
batchSize = 16
num_train_steps = totalEpoch
WU = 0.2
num_training_steps = ((len(data_id)/batchSize)+1) * totalEpoch
num_warmup_steps = int(num_training_steps * WU)
#scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)
optimizer = AdamW(model.parameters(), lr = lr, eps = eps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_train_steps, num_training_steps=num_training_steps)
loss_fn = nn.MSELoss()
#loss_fn = nn.CrossEntropyLoss()##Task2
device = torch.device("cuda")
model.to(device)
for epoch in range(totalEpoch):
  batchStart = 0
  train_loss = 0
  optimizer.zero_grad()
  while batchStart <= len(data_meanGrade):
    batchEnd = batchStart + batchSize
    if batchEnd > len(data_meanGrade):
      batchEnd = len(data_meanGrade)
    old_title = data_orginal[batchStart:batchEnd]
    new_title = data_replaced[batchStart:batchEnd]
    encoding_old = tokenizer(old_title, return_tensors='pt', padding=True, truncation=True)
    encoding_new = tokenizer(new_title, return_tensors='pt', padding=True, truncation=True)
    input_ids_old = encoding_old['input_ids'].to(device)
    input_ids_new = encoding_new['input_ids'].to(device)
    attention_mask_old = encoding_old['attention_mask'].to(device)
    attention_mask_new = encoding_new['attention_mask'].to(device)
    labels = torch.tensor(data_meanGrade[batchStart:batchEnd]).to(device)
    model.train()
    #outputs = model(input_ids_new, input_ids_old, attention_mask_new, attention_mask_old)[0]
    outputs = model(input_ids_new, attention_mask_new, input_ids_old, attention_mask_old)
    #loss = torch.sqrt(loss_fn(outputs,labels))
    loss = torch.sqrt(((outputs - labels)**2).mean())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    batchStart = batchStart + batchSize
    
  with torch.no_grad():
    new_title = val_replaced
    old_title = val_orginal
    encoding_old = tokenizer(old_title, return_tensors='pt', padding=True, truncation=True)
    encoding_new = tokenizer(new_title, return_tensors='pt', padding=True, truncation=True)
    input_ids_old = encoding_old['input_ids'].to(device)
    input_ids_new = encoding_new['input_ids'].to(device)
    attention_mask_old = encoding_old['attention_mask'].to(device)
    attention_mask_new = encoding_new['attention_mask'].to(device)
    #outputs = model(input_ids_new, input_ids_old, attention_mask_new, attention_mask_old)[0]
    outputs = model(input_ids_new, attention_mask_new, input_ids_old, attention_mask_old)
    labels = torch.tensor(val_meanGrade).to(device)
    loss = torch.sqrt(loss_fn(outputs,labels))
    valid_loss = loss.item()
    print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')
with torch.no_grad():
  new_title = test_replaced
  old_title = test_orginal
  encoding_old = tokenizer(old_title, return_tensors='pt', padding=True, truncation=True)
  encoding_new = tokenizer(new_title, return_tensors='pt', padding=True, truncation=True)
  input_ids_old = encoding_old['input_ids'].to(device)
  input_ids_new = encoding_new['input_ids'].to(device)
  attention_mask_old = encoding_old['attention_mask'].to(device)
  attention_mask_new = encoding_new['attention_mask'].to(device)
  #outputs = model(input_ids_new, input_ids_old, attention_mask_new, attention_mask_old)[0]
  outputs = model(input_ids_new, attention_mask_new, input_ids_old, attention_mask_old)
  labels = torch.tensor(test_meanGrade).to(device)
  loss = torch.sqrt(loss_fn(outputs,labels))
  valid_loss = loss.item()
  print(f'| Val. Loss: {valid_loss:.3f}')


##CLASS

In [ ]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from tokenizers import AddedToken
from transformers import RobertaTokenizer
import math
import csv
import re
import os
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn import model_selection
from sklearn import metrics
import transformers
import tokenizers
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import RobertaConfig, RobertaModel
#from transformers.modeling_roberta import RobertaClassificationHead
from sklearn.model_selection import train_test_split
from tqdm.autonotebook import tqdm
import utils
from transformers import RobertaConfig, RobertaModel, RobertaForSequenceClassification, RobertaTokenizer
from transformers import BertConfig, BertModel, BertForSequenceClassification, BertTokenizer
from transformers import AdamW
import transformers
import torch
from transformers import get_linear_schedule_with_warmup
import zipfile
import random
def fix_seed(seed=1):
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  np.random.seed(seed)
  random.seed(seed)
fix_seed()
configuration = RobertaConfig()
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=True)
dev_type = 'No_dev'
import csv
import re
def readTask1Data(data_address):
  data_id = list()
  data_mask = list()
  data_orginal = list()
  data_replaced = list()
  data_grades = list()
  data_meanGrade = list()
  data_editword = list()
  data_orginalword = list()
  flag = 0
  p = r"(?<=<).+?(?=/>)" 
  with open(data_address, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      data_id.append(row[0])
      line = row[1]
      while '  ' in line:
        line.replace('  ',' ')
      line = line.replace('/ >','/>')
      line = line.replace('< ','<')
      line = line.replace(' />','/>')
      data_orginalword.append(re.findall(p, line))
      data_orginal_temp = line.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal.append(data_orginal_temp)
      data_mask_temp = re.sub(p, 'edit', line)
      if '<edit/>' not in data_mask_temp:
        print(data_mask_temp)
      data_mask.append(data_mask_temp)
      data_replaced_temp = data_mask_temp.replace('<edit/>',row[2])
      data_replaced.append(data_replaced_temp)
      data_grades.append(row[3])
      data_meanGrade.append([float(row[4])])
      data_editword.append(row[2])
  return  data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test_mask, test_orginal, test_replaced, test_grades, test_meanGrade, test_editword, test_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
def splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, valid_num=0.3):
  train_id = list()
  train_mask = list()
  train_orginal = list()
  train_replaced = list()
  train_grades = list()
  train_meanGrade = list()
  train_editword = list()
  train_orginalword = list()
  val_id = list()
  val_mask = list()
  val_orginal = list()
  val_replaced = list()
  val_grades = list()
  val_meanGrade = list()
  val_editword = list()
  val_orginalword = list()
  temp_list = list()
  for num in range(len(data_id)):
    temp_list.append([data_id[num], data_mask[num], data_orginal[num], data_replaced[num], data_grades[num], data_meanGrade[num], data_editword[num], data_orginalword[num]])
  train_set,valid_set = train_test_split(temp_list,test_size = valid_num)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in train_set:
    train_id.append(set_id_temp)
    train_mask.append(set_mask_temp)
    train_orginal.append(set_orginal_temp)
    train_replaced.append(set_replaced_temp)
    train_grades.append(set_grades_temp)
    train_meanGrade.append(set_meanGrade_temp)
    train_editword.append(set_editword_temp)
    train_orginalword.append(set_orginalword_temp)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in valid_set:
    val_id.append(set_id_temp)
    val_mask.append(set_mask_temp)
    val_orginal.append(set_orginal_temp)
    val_replaced.append(set_replaced_temp)
    val_grades.append(set_grades_temp)
    val_meanGrade.append(set_meanGrade_temp)
    val_editword.append(set_editword_temp)
    val_orginalword.append(set_orginalword_temp)
  return train_id, train_mask, train_orginal, train_replaced, train_grades, train_meanGrade, train_editword, train_orginalword, val_id,val_mask,val_orginal,val_replaced,val_grades,val_meanGrade,val_editword,val_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, val_id, val_mask, val_orginal, val_replaced, val_grades, val_meanGrade, val_editword, val_orginalword = splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword)
dev_type = 'No_dev'
import csv
import re
def readTask1Data(data_address):
  data_id = list()
  data_mask = list()
  data_orginal = list()
  data_replaced = list()
  data_grades = list()
  data_meanGrade = list()
  data_editword = list()
  data_orginalword = list()
  flag = 0
  p = r"(?<=<).+?(?=/>)" 
  with open(data_address, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      data_id.append(row[0])
      line = row[1]
      while '  ' in line:
        line.replace('  ',' ')
      line = line.replace('/ >','/>')
      line = line.replace('< ','<')
      line = line.replace(' />','/>')
      data_orginalword.append(re.findall(p, line))
      data_orginal_temp = line.replace('<','').replace('/>','').replace('  ',' ')
      data_orginal.append(data_orginal_temp)
      data_mask_temp = re.sub(p, 'edit', line)
      if '<edit/>' not in data_mask_temp:
        print(data_mask_temp)
      data_mask.append(data_mask_temp)
      data_replaced_temp = data_mask_temp.replace('<edit/>',row[2])
      data_replaced.append(data_replaced_temp)
      data_grades.append(row[3])
      data_meanGrade.append([float(row[4])])
      data_editword.append(row[2])
  return  data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/train.csv')
test_id, test_mask, test_orginal, test_replaced, test_grades, test_meanGrade, test_editword, test_orginalword = readTask1Data('/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv')
def splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, valid_num=0.3):
  train_id = list()
  train_mask = list()
  train_orginal = list()
  train_replaced = list()
  train_grades = list()
  train_meanGrade = list()
  train_editword = list()
  train_orginalword = list()
  val_id = list()
  val_mask = list()
  val_orginal = list()
  val_replaced = list()
  val_grades = list()
  val_meanGrade = list()
  val_editword = list()
  val_orginalword = list()
  temp_list = list()
  for num in range(len(data_id)):
    temp_list.append([data_id[num], data_mask[num], data_orginal[num], data_replaced[num], data_grades[num], data_meanGrade[num], data_editword[num], data_orginalword[num]])
  train_set,valid_set = train_test_split(temp_list,test_size = valid_num)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in train_set:
    train_id.append(set_id_temp)
    train_mask.append(set_mask_temp)
    train_orginal.append(set_orginal_temp)
    train_replaced.append(set_replaced_temp)
    train_grades.append(set_grades_temp)
    train_meanGrade.append(set_meanGrade_temp)
    train_editword.append(set_editword_temp)
    train_orginalword.append(set_orginalword_temp)
  for (set_id_temp, set_mask_temp, set_orginal_temp, set_replaced_temp, set_grades_temp, set_meanGrade_temp, set_editword_temp, set_orginalword_temp) in valid_set:
    val_id.append(set_id_temp)
    val_mask.append(set_mask_temp)
    val_orginal.append(set_orginal_temp)
    val_replaced.append(set_replaced_temp)
    val_grades.append(set_grades_temp)
    val_meanGrade.append(set_meanGrade_temp)
    val_editword.append(set_editword_temp)
    val_orginalword.append(set_orginalword_temp)
  return train_id, train_mask, train_orginal, train_replaced, train_grades, train_meanGrade, train_editword, train_orginalword, val_id,val_mask,val_orginal,val_replaced,val_grades,val_meanGrade,val_editword,val_orginalword
data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword, val_id, val_mask, val_orginal, val_replaced, val_grades, val_meanGrade, val_editword, val_orginalword = splitTrainValid(data_id, data_mask, data_orginal, data_replaced, data_grades, data_meanGrade, data_editword, data_orginalword)
model = RobertaForSequenceClassification.from_pretrained('roberta-base', return_dict=True, num_labels=1)
model_name = 'OrginalRoBERTaSequenceClassification'
lr = 3e-5
max_grad_norm = 1.0
eps = 1e-8
batchSize = 16
num_train_steps = totalEpoch
WU = 0.2
num_training_steps = ((len(data_id)/batchSize)+1) * totalEpoch
num_warmup_steps = int(num_training_steps * WU)
#scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_train_steps)
optimizer = AdamW(model.parameters(), lr = lr, eps = eps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_train_steps, num_training_steps=num_training_steps)
loss_fn = nn.MSELoss()
#loss_fn = nn.CrossEntropyLoss()##Task2
device = torch.device("cuda")
model.to(device)
for epoch in range(totalEpoch):
  batchStart = 0
  train_loss = 0
  optimizer.zero_grad()
  while batchStart <= len(data_meanGrade):
    batchEnd = batchStart + batchSize
    if batchEnd > len(data_meanGrade):
      batchEnd = len(data_meanGrade)
    input_title = data_replaced[batchStart:batchEnd]
    input_word = data_editword[batchStart:batchEnd]
    input_data = [[input_title[num_],input_word[num_]] for num_ in range(len(input_title))]
    encoding = tokenizer(input_data, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    labels = torch.tensor(data_meanGrade[batchStart:batchEnd]).to(device)
    model.train()
    outputs = model(input_ids, attention_mask=attention_mask)[0]
    #outputs = model(input_ids, attention_mask=attention_mask)
    loss = torch.sqrt(loss_fn(outputs,labels))
    #loss = 2*torch.sqrt(((outputs - labels)**2).mean())
    train_loss += loss.item()
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    batchStart = batchStart + batchSize
    
  with torch.no_grad():
    input_title = val_replaced
    input_word = val_editword
    input_data = [[input_title[num_],input_word[num_]] for num_ in range(len(input_title))]
    encoding = tokenizer(input_data, return_tensors='pt', padding=True, truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)[0]
    #outputs = model(input_ids, attention_mask=attention_mask)
    labels = torch.tensor(val_meanGrade).to(device)
    loss = torch.sqrt(loss_fn(outputs,labels))
    valid_loss = loss.item()
    print(f'| Epoch: {epoch:02} | Train Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')
with torch.no_grad():
  input_title = test_replaced
  input_word = test_editword
  input_data = [[input_title[num_],input_word[num_]] for num_ in range(len(input_title))]
  encoding = tokenizer(input_data, return_tensors='pt', padding=True, truncation=True)
  input_ids = encoding['input_ids'].to(device)
  attention_mask = encoding['attention_mask'].to(device)
  outputs = model(input_ids, attention_mask=attention_mask)[0]
  #outputs = model(input_ids, attention_mask=attention_mask)
  labels = torch.tensor(test_meanGrade).to(device)
  loss = torch.sqrt(loss_fn(outputs,labels))
  valid_loss = loss.item()
  print(f'| Val. Loss: {valid_loss:.3f}')

#Verify

In [ ]:
import csv
import os
import math
true_address = '/content/drive/MyDrive/Colab Notebooks/semeval-2020-task-7-dataset/subtask-1/test.csv'
flag = 0
true_dict = dict()
predict_dict = dict()
with open(true_address, 'r') as f:
  reader = csv.reader(f)
  for row in reader:
    if flag == 0:
      flag = 1
      continue
    true_dict[row[0]] = float(row[4])
folder = '/content/drive/MyDrive/Colab Notebooks/FinalProject/result'
csv_file = dict()
import os
for file in os.listdir(folder):
    if file.endswith(".csv"):
      print(file)
      csv_file[file] = os.path.join(folder, file)
ff = open('result.csv','w',encoding='utf-8')
str_ = 'Method,MAE,MSE,RMSE,MEAN,R_quared\n'
ff.write(str_)
for key in csv_file:
  MAE = 0.0
  MSE = 0.0
  RMSE = 0.0
  flag = 0
  MEAN = 0.0
  R_quared = 0.0

  with open(csv_file[key], 'r') as f:
    reader = csv.reader(f)
    for row in reader:
      if flag == 0:
        flag = 1
        continue
      predict_dict[row[0]] = float(row[1])
  for IDs in predict_dict:
    MAE = MAE + abs(predict_dict[IDs] - true_dict[IDs])
    RMSE = RMSE + (predict_dict[IDs] - true_dict[IDs]) ** 2
    MSE = MSE + (predict_dict[IDs] - true_dict[IDs]) ** 2
    MEAN = MEAN + predict_dict[IDs] 
  MAE = MAE/len(predict_dict)
  RMSE = math.sqrt(RMSE/len(predict_dict))
  MSE = MSE/len(predict_dict)
  MEAN = MEAN/len(predict_dict)
  R_quared_top = 0.0
  R_quared_bottom = 0.0
  for IDs in predict_dict:
    R_quared_top = R_quared_top + (predict_dict[IDs] - true_dict[IDs]) ** 2
    R_quared_bottom = R_quared_bottom + (MEAN - true_dict[IDs]) ** 2
  R_quared = 1-(R_quared_top/R_quared_bottom)
  str_ = key.replace('.csv','') + ',' + str(MAE) + ',' + str(MSE) + ',' + str(RMSE) + ',' + str(MEAN) + ',' + str(R_quared) +'\n'
  ff.write(str_)
ff.close()

In [ ]:
R_quared

In [ ]:
RMSE